<a href="https://colab.research.google.com/github/Datarian723/AutoHPO/blob/main/AutoHPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TODO

In [1]:
"""
(1)FIGURE_OUT: WHY ON EARTH Wrapper's retrieval of 'dls_func' is returning [get_sig] INSTEAD of get_sig.
   MODIFY: UNTIL Wrapper Testing passes.
(2)MODIFY: HpoTrainer, Auto.
   TEST: The entire Auto-HPO process.
(3)START: The new Kaggle competition(check your recent Gmail notification about it)
   TEST: The entire Auto-HPO process ON: 3D Image reconstruction Challenge
   TEST: The entire Auto-HPO process ON: 3D Image reconstruction Challenge + CapsuleNetwork
"""

"\n(1)FIGURE_OUT: WHY ON EARTH Wrapper's retrieval of 'dls_func' is returning [get_sig] INSTEAD of get_sig.\n   MODIFY: UNTIL Wrapper Testing passes.\n(2)MODIFY: HpoTrainer, Auto.\n   TEST: The entire Auto-HPO process.\n(3)START: The new Kaggle competition(check your recent Gmail notification about it)\n   TEST: The entire Auto-HPO process ON: 3D Image reconstruction Challenge\n   TEST: The entire Auto-HPO process ON: 3D Image reconstruction Challenge + CapsuleNetwork\n"

#Prep

In [3]:
# ---------------------------
# Special Imports
# ---------------------------
from __future__ import annotations  # Enable forward references for type hints

# ---------------------------
# Standard Libraries
# ---------------------------
import os  # Operating system utilities
import gc  # Garbage collection for memory management
import logging  # Logging utilities
import sys  # System-specific parameters and functions
import unittest  # For unit-testing Python code
import copy  # For deep and shallow copying of objects
import math  # Math operations
import random  # Random number generation
import gzip  # For compressing and decompressing data
from functools import partial  # Partial function application
from collections import defaultdict  # Dictionary subclass with default values
from sortedcontainers import SortedSet  # Sorted set implementation
from unittest.mock import Mock, patch  # For mocking and patching in unit tests
from inspect import signature  # Inspecting callable signatures
from typing import Callable, Union  # Type annotations for type hints
from ast import Is  # Abstract syntax tree utilities
from abc import ABC, abstractmethod  # Abstract Base Classes for inheritance

# ---------------------------
# Scientific Libraries
# ---------------------------
import numpy as np  # Array and matrix operations
import pandas as pd  # Data manipulation and analysis
import matplotlib.pyplot as plt  # Plotting and visualization

# ---------------------------
# PyTorch and Related Modules
# ---------------------------
import torch  # Core PyTorch library for tensor operations
import torch.nn as nn  # Neural network modules
import torch.nn.functional as F  # Functional API for neural networks
import torchvision.models as models  # Pre-trained models from torchvision
from torch.multiprocessing import Pool, set_start_method  # Multiprocessing utilities for PyTorch

# ---------------------------
# FastAI Libraries
# ---------------------------
from fastai.vision.all import *  # All-in-one import for vision-specific modules
from fastai.metrics import *  # Evaluation metrics for model performance
from fastai.callback.hook import *  # Hooks for capturing intermediate model states
from fastai.callback.tracker import *  # Training callback trackers (e.g., early stopping)
from fastai.learner import Learner  # Core Learner object for model training

# ---------------------------
# Optuna for Hyperparameter Optimization
# ---------------------------
import optuna  # Core Optuna library for optimization
from optuna.integration import FastAIPruningCallback  # Pruning callback for FastAI integration
from optuna.trial import Trial, FrozenTrial  # Trial classes for defining and managing trials

# ---------------------------
# Google Colab Utilities
# ---------------------------
from google.colab import files  # File utilities for Colab
from google.colab import drive  # Mounting Google Drive in Colab

#Class: Var

In [4]:
class Var:
    def __init__(self, var_type: str, source: dict):
        #Extract distribution, sample_method for error checking.
        sample_method = source['sample']
        distribution = source['params']['choices'] if sample_method=='categorical' else source['params']
        #ERROR CHECK:
        self._error_check(distribution, sample_method)

        #Member initializations:
        self.var_type = var_type
        self.sample_method = sample_method
        #frozen_dist is used for value conversion, in case value NOT currently in 'self.get_dist()' is requested for conversion.
        self._frozen_dist = distribution.copy()
        self._dist = list(range(len(self._frozen_dist))) if sample_method=='categorical' else distribution.copy()
        #fixed <- IF (categorical distribution has one element) OR (low==high)
        self.fixed = (self.sample_method=='categorical' and len(self._dist)==1) or (self.sample_method!='categorical' and self._dist['low']==self._dist['high'])

    def _error_check(self, distribution, sample_method: str):
        if not isinstance(distribution, (list,dict)):
            raise ValueError(f"Input 'distribution' MUST be of type in (list,dict) but found: {type(distribution)}")
        if sample_method not in ['categorical', 'float', 'int']:
            raise ValueError(f"sample method MUST be one of ['categorical', 'float', 'int'] but found: {sample_method}")
        if sample_method!='categorical' and not isinstance(distribution, dict) or sample_method=='categorical' and not isinstance(distribution, list):
            raise ValueError(f"sample_method: {sample_method} is not compatible with distribution: {distribution}. Categorical sampling requires a list, rest requires a dictionary")
        cat_valid_params = sample_method!='categorical' or len(distribution)
        non_cat_valid_params = sample_method=='categorical' or (distribution.get('low', 0) <= distribution.get('high', -1))
        if not cat_valid_params or not non_cat_valid_params:
            msg = f"Categorical distribution needs at least one value in it: {distribution}" if sample_method=='categorical' else f"distribution needs (low,high) as keys and distribution[low]<=distribution[high] but found: {distribution}"
            raise ValueError(msg)

    def __hash__(self):
        return hash((self.var_type, self.sample_method))

    def __eq__(self, other: Var):
        """Two Var objects are considered 'equal'
           IFF (1)other is Var object
               (2)self.var_type == other.var_type
               (3)self.params == other.params
        """
        return all([
            isinstance(other, Var),
            self.var_type == other.var_type,
            self.sample_method == other.sample_method,
            self._frozen_dist == other._frozen_dist,
            self._dist == other._dist
        ])

    def __str__(self):
        if self.sample_method == 'categorical':
            config_space = f"'choices' = {[self._frozen_dist[idx] for idx in self._dist]}"
        else:
            config_space = (
                f"'low': {self._dist['low']}, "
                f"'high': {self._dist['high']}, "
                f"'log': {self._dist.get('log', False)}, "
                f"'step': {self._dist.get('step', 'unspecified')}"
            )
        output = f"Var of type: {self.var_type}\nConfiguration Space:\n{config_space}\n"
        return output

    def contain_idx(self, idx, current: bool = False):
        if self.sample_method!='categorical' and type(idx)!=eval(self.sample_method) or self.sample_method=='categorical' and type(idx)!=int:
            return False
        if self.sample_method!='categorical':
            dist = self._dist if current else self._frozen_dist
            return dist['low'] <= idx <= dist['high']
        return idx in self._dist if current else 0 <= idx < len(self._frozen_dist)

    def contain_val(self, val, current: bool = False):
        if self.sample_method != 'categorical':
            return self.contain_idx(idx=val, current=current)
        if self.var_type=='action_func' and isinstance(val, Action):
            val = val.action_func
        return val in self.distribution() if current else val in self._frozen_dist

    def convert_idx_to_val(self, idx):
        if not self.contain_idx(idx=idx, current=False):
            raise ValueError(f"Input index 'idx' is NOT a proper index for conversion via Var:{self.var_type}'s distribution: {list(range(len(self._frozen_dist))) if self.sample_method=='categorical' else self._frozen_dist}")
        if self.sample_method!='categorical':
            return idx
        return self._frozen_dist[idx]

    def convert_val_to_idx(self, val):
        if not self.contain_val(val=val, current=False):
            raise ValueError(f"Input value 'val': {val} is NOT a proper value for conversion via Var:{self.var_type}'s distribution: {self._frozen_dist}")
        if self.sample_method!='categorical':
            return val
        return self._frozen_dist.index(val)

    def distribution(self, indices: bool = False):
        if self.sample_method!='categorical':
            return self._dist
        if indices:
            return self._dist
        return [self._frozen_dist[idx] for idx in self._dist]

    def sample(self, trial):
        if not isinstance(trial, (Trial, FrozenTrial)):
            raise ValueError(f"Input 'trial' must be of type in (optuna::Trial, optuna::FrozenTrial) but found: {trial}")

        if isinstance(trial, FrozenTrial):
            idx_or_val = trial.params[self.var_type]
            #sample_method=='categorical' <-> retrieved value is an index NOT an actual value.
            sampled_val = self.convert_idx_to_val(idx_or_val) if self.sample_method=='categorical' else idx_or_val
            return sampled_val

        sampled_val = None
        if self.sample_method == 'categorical':
            idx = trial.suggest_categorical(self.var_type, choices=self._dist)
            sampled_val = self._frozen_dist[idx]
        elif self.sample_method == 'float':
            sampled_val = trial.suggest_float(self.var_type, **self._dist)
        elif self.sample_method == 'int':
            sampled_val = trial.suggest_int(self.var_type, **self._dist)

        return sampled_val

    def update(self, vals: list):
        #Proior processing:
        if not len(vals):
            return
        if not all(isinstance(val, (int,float)) for val in vals):
            raise ValueError(f"Var of type: {self.var_type} with sample_method: {self.sample_method} recieved update request with 'vals': {vals} which contain element(s) NOT of type in (int,float)")
        vals = sorted(vals.copy()) if self.sample_method=='categorical' else vals

        set_vals = set(vals)
        if len(set_vals)==1:
            vals = [vals[0]]
            self.fixed = True

        #ERROR CHECKS:
        if self.sample_method!='categorical' and (min(vals)<self._dist['low'] or max(vals)>self._dist['high']):
            raise ValueError(f"Var of type: {self.var_type} with sample_method: {self.sample_method} recieved update request with 'vals': {vals} NOT in valid range: [{self._dist['low']},{self._dist['high']}]. Note the initial search space: [{self._frozen_dist['low']},{self._frozen_dist['high']}].")
        if self.sample_method=='categorical' and not set_vals.issubset(set(self._dist)):
            raise ValueError(f"Var of type: {self.var_type} with sample_method: {self.sample_method} recieved update request with 'vals': {vals} NOT in valid range: {self._dist}. Note the initial search space: {self._frozen_dist}")

        #UPDATE: configuration space.
        if self.sample_method=='categorical':
            self._dist = vals
        else:
            self._dist['low'] = min(vals)
            self._dist['high'] = max(vals)

#Test: Var

In [5]:
class TestVar(unittest.TestCase):

    def setUp(self):
        self.categorical_source = {
            'params': {'choices': [1, 2, 3, 4, 5]},
            'sample': 'categorical'
        }
        self.float_source = {
            'params': {'low': 0.1, 'high': 1.0},
            'sample': 'float'
        }
        self.int_source = {
            'params': {'low': 1, 'high': 10},
            'sample': 'int'
        }

    def test_init_categorical(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertEqual(var.var_type, 'test_var')
        self.assertEqual(var.sample_method, 'categorical')
        self.assertEqual(var._frozen_dist, [1, 2, 3, 4, 5])
        self.assertEqual(var._dist, [0, 1, 2, 3, 4])
        self.assertFalse(var.fixed)

    def test_init_float(self):
        var = Var(var_type='test_var', source=self.float_source)
        self.assertEqual(var.var_type, 'test_var')
        self.assertEqual(var.sample_method, 'float')
        self.assertEqual(var._frozen_dist, {'low': 0.1, 'high': 1.0})
        self.assertEqual(var._dist, {'low': 0.1, 'high': 1.0})
        self.assertFalse(var.fixed)

    def test_init_int(self):
        var = Var(var_type='test_var', source=self.int_source)
        self.assertEqual(var.var_type, 'test_var')
        self.assertEqual(var.sample_method, 'int')
        self.assertEqual(var._frozen_dist, {'low': 1, 'high': 10})
        self.assertEqual(var._dist, {'low': 1, 'high': 10})
        self.assertFalse(var.fixed)

    def test_init_error_invalid_sample_method(self):
        with self.assertRaises(ValueError):
            Var(var_type='test_var', source={'params': {'choices': [1, 2]}, 'sample': 'invalid'})

    def test_init_error_invalid_distribution_type(self):
        with self.assertRaises(ValueError):
            Var(var_type='test_var', source={'params': {'choices': {'key': 'value'}}, 'sample': 'categorical'})

    def test_contain_idx(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertTrue(var.contain_idx(2))
        self.assertFalse(var.contain_idx(5))

        var_float = Var(var_type='test_var', source=self.float_source)
        self.assertTrue(var_float.contain_idx(0.5))
        self.assertFalse(var_float.contain_idx(1.1))

    def test_contain_val(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertTrue(var.contain_val(3))
        self.assertFalse(var.contain_val(6))

        var_float = Var(var_type='test_var', source=self.float_source)
        self.assertTrue(var_float.contain_val(0.5))
        self.assertFalse(var_float.contain_val(1.1))

    def test_convert_idx_to_val(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertEqual(var.convert_idx_to_val(2), 3)

        var_float = Var(var_type='test_var', source=self.float_source)
        self.assertEqual(var_float.convert_idx_to_val(0.5), 0.5)

    def test_convert_val_to_idx(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertEqual(var.convert_val_to_idx(3), 2)

        with self.assertRaises(ValueError):
            var.convert_val_to_idx(6)

    def test_distribution(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertEqual(var.distribution(), [1, 2, 3, 4, 5])

        var_float = Var(var_type='test_var', source=self.float_source)
        self.assertEqual(var_float.distribution(), {'low': 0.1, 'high': 1.0})

    def test_sample_categorical(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        trial = optuna.create_study(direction='minimize').ask()
        trial.suggest_categorical = Mock(return_value=2)
        sampled_val = var.sample(trial)
        self.assertEqual(sampled_val, 3)

    def test_sample_float(self):
        var = Var(var_type='test_var', source=self.float_source)
        trial = optuna.create_study(direction='minimize').ask()
        trial.suggest_float = Mock(return_value=0.5)
        sampled_val = var.sample(trial)
        self.assertEqual(sampled_val, 0.5)

    def test_sample_error(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        with self.assertRaises(ValueError):
            var.sample('not_a_trial')

    def test_update_categorical(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        var.update([1, 2])
        self.assertEqual(var.distribution(indices=True), [1,2])
        self.assertEqual(var.distribution(), [2,3])
        self.assertFalse(var.fixed)

    def test_update_float(self):
        var = Var(var_type='test_var', source=self.float_source)
        var.update([0.2, 0.9])
        self.assertEqual(var._dist['low'], 0.2)
        self.assertEqual(var._dist['high'], 0.9)

    def test_update_error_out_of_range(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        with self.assertRaises(ValueError):
            var.update([0, 6])

    def test_str(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        expected_str = "Var of type: test_var\nConfiguration Space:\n'choices' = [1, 2, 3, 4, 5]\n"
        self.assertEqual(str(var), expected_str)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..........[I 2024-09-27 21:37:26,523] A new study created in memory with name: no-name-54b7dbbc-9bd5-4a9c-bd2a-7242277ad5b2
..[I 2024-09-27 21:37:26,529] A new study created in memory with name: no-name-5350e2d4-5ac0-4fb0-a06e-00bacf8ef38c
.....
----------------------------------------------------------------------
Ran 17 tests in 0.063s

OK


#Class: Action

In [5]:
class Action:
    def __init__(self, action_func: Callable, sig_dict: dict):
        if not all(isinstance(var,Var) for var in sig_dict.values()):
            raise ValueError(f"Constructor of object 'Action' with action_func: '{action_func.__name__}' received an invalid dictionary sig_dict: {sig_dict}")
        self.action_func = action_func
        self.sig_dict = sig_dict
        self.sig = frozenset(sig_dict.keys())
        self.params = {}
        self.action_state = None
        self._model_loader = None

    def __len__(self):
        return len(self.sig_dict)

    def __hash__(self):
        return hash(self.sig)

    def __eq__(self, other: Action):
        """Two Action objects are considered 'equal'
        IFF (1)other is Action object
            (2)self.action_func == other.action_func
            (3)self.params == other.params
            (4)self.action_state == other.action_state
        """
        self_params = self._model_loader.keywords if self._model_loader is not None else self._model_loader
        other_params = other._model_loader.keywords if other._model_loader is not None else other._model_loader
        if self_params != other_params:
            return False

        # Compare action states if both are dictionaries
        if isinstance(self.action_state, dict) and isinstance(other.action_state, dict):
            if self.action_state.keys() != other.action_state.keys():
                return False
            if not all(torch.equal(self.action_state[key], other.action_state[key]) for key in self.action_state.keys()):
                return False
        elif self.action_state != other.action_state: #IF one is None and the other is not None.
            return False

        return all([
            self.action_func == other.action_func,
            self.sig_dict == other.sig_dict,
        ])


    def __str__(self):
        signature = " ".join([f"{var}" for var in self.sig_dict.values()])
        output = f"Action of func: {self.action_func.__name__}\nSignature Vars:\n{signature}\n"
        if self._model_loader:
            parameters = " ".join([f"{key}={val}" for key,val in self._model_loader.keywords.items()])
            output += f"Model loader parameters: {parameters}\n"
        return output

    def get_var(self, var_type):
        if var_type not in self.keys():
            raise ValueError(f"'{var_type}' is not a valid signature key for Action: '{self.action_func.__name__}'. Available keys: {set(self.keys())}")
        return self.sig_dict[var_type]

    def keys(self):
        return self.sig_dict.keys()

    def values(self):
        return self.sig_dict.values()

    def items(self):
        return self.sig_dict.items()

    def is_loadable(self):
        return self._model_loader is not None

    def is_fixed(self):
        return all(var.fixed for var in self.sig_dict.values())

    def parameterize(self, trial: optuna.Trial):
        """
        ACCEPTS:
        trial := Active optuna trial objet.
                 IF input 'manual_action_params' is not empty, THEN 'trial' will NOT be used.
        RETURNS: void.
                 The function parameterizes the caller Action object.
                 Without any saved action_state, the caller Action object needs to be parameterized every time before .__call__()
        """
        #ERROR CHECKS:
        if self.action_state is not None and not self.params:
            raise ValueError(f"Action: {self.action_func.__name__} has a saved action state(self.action_state!=None) but NO saved params(self.params==None).")
        if self.action_state is None and trial is None:
            raise ValueError(f"Action: {self.action_func.__name__} has no saved action state(self.action_state==None) and no input trial passed in(self.trial==None). No sampling can be done.")

        if self.action_state is None:
            self.params = {cat:var.sample(trial=trial) for cat,var in self.items()}

        #self._model_loader <- Fully parameterized model with self.params
        #NOTE: IF it has a saved action state, the consistent self.params will restore the recenmost state(based on the recentmost top sampling results)
        self._model_loader = partial(self.action_func, **self.params)

    def __call__(self, device: torch.device):
        if self.action_state is not None and not self.params:
            raise ValueError(f"Action: {self.action_func.__name__} has a saved action state(self.action_state!=None) but no saved params exists(self.params empty). Likely an internal logic issue")
        if self._model_loader is None:
            raise ValueError(f"Action: {self.action_func.__name__} cannot be called. Call Action.parameterize() first and then try it again.")

        #model <- Load the model onto 'device'.
        model = self._model_loader(device=device)

        #IF 'self.action_state' exists, THEN load the saved weights from the state dictionary(=self.action_state) to the model
        if self.action_state:
            try:
                name = self.action_func.__name__
                model.load_state_dict(self.action_state)
            except torch.nn.modules.module.ModuleAttributeError as e:
                print(f"Attribute error: {e}. The state dictionary of Action: '{name}' may not match the model.")
            except RuntimeError as e:
                print(f"Runtime error: {e}. The state dictionary of Action: '{name}' may have unmatching layer sizes or missing keys.")
            except Exception as e:
                print(f"An unexpected error occurred while loading state dictionary to Action '{name}': {e}")

        #Last but not least, return the model.
        return model

    def update_sig(self, update_params: dict):
        #IF Action (1)has a saved 'action_state' OR (2)0 elements in the 'update_params'
        #THEN NO update shall be done.
        if self.action_state or not len(update_params):
            return

        #ERROR CHECK: Make sure that ALL the keys in 'self.sig_dict' belong to 'sig_params'
        leftovers = set(self.keys()) - set(update_params.keys())
        if leftovers:
            raise ValueError(f"Updating Action: {self.action_func.__name__} requires following missing keys in 'sig_params': {leftovers}")
        if not all(isinstance(val,list) for val in update_params.values()):
            raise ValueError(f"Updating Action: {self.action_func.__name__} with input 'update_params' failed. ALL the values in 'update_params' MUST be of type 'list' but found: {update_params}")

        #Update: signature configuration space(=='self.sig_dict')
        for cat,var in self.items():
            var.update(vals=update_params[cat])

    def update_state(self, obj):
        """
        profile := A dictionary that contains the caller's signature(except 'device') as its subset.
                   Additionally, every (sig->value) must contain the same value as what initialized the passed in 'obj'.
        """
        #ERROR CHECK1: Type of obj.
        if not isinstance(obj, (nn.Module, Learner, dict)):
            raise ValueError(f"Updating action state of Action: {self.action_func.__name__} failed. Input object 'obj' must be of type in (nn.Module, Learner, dict) but found: {type(obj)}")
        #ERROR CHECK2: This catches if one attempts to update the Action object with overriden
        if self.action_state is not None and self.params != self._model_loader.keywords:
            raise ValueError(f"Updating action state of Action: {self.action_func.__name__} failed. Make sure that this object was sampled with valid Trialin the previuos HPO run. If .get_loadable_action() was used, then .update_state is NOT allowed.")

        if isinstance(obj, dict):
            self.action_state = obj
        else:
            self.action_state = obj.model.state_dict() if isinstance(obj, Learner) else obj.state_dict()

        #Manually set 'self.params'. This way, saved action_state -> self.params available!
        self.params = self._model_loader.keywords.copy()

#Test: Action

In [7]:
"""
Parameters used in the testin of class 'Action' defined in this cell
"""
class ModelDefault(nn.Module):
    def __init__(self, rand_number: int):
        super().__init__()
        self.net = nn.Linear(16,10)
        self.rand_number = torch.tensor(rand_number)

    def forward(self, x):
        return self.net(x) + self.rand_number

class ModelDropout(nn.Module):
    def __init__(self, activation: nn.Module, dropout_p: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features=16,out_features=10),
            activation(),
            nn.Dropout(p=dropout_p),
            nn.Linear(in_features=10,out_features=10)
        )

    def forward(self, x):
        return self.net(x)

def action_default(rand_number: int, device: torch.device):
    return ModelDefault(rand_number=rand_number).to(device)

def action_dropout(activation, dropout_p, device: torch.device):
    #model_dropout := Class header of 'ModelDropout'
    return ModelDropout(activation=activation, dropout_p=dropout_p).to(device)

#ASSUMED: Input size of 10x16. (Single batch == 10 (4x4 flattened vector)s )
#FIRST_LAYER: Fully connected linear layer of size 16x10
#ACTIVATION: One of nn.ReLU, nn.PReLU, nn.SiLU
#DROPOUT: nn.Dropout(p=dropout_p)
#OUT: .shape == [10,10]
#NOTE: Input 'model: Callable' will be the class header 'Model'.

In [8]:
class TestAction(unittest.TestCase):
    def setUp(self):
        default_sig_dict = {
            'rand_number': Var(var_type='rand_number', source={'params': {'choices': list(range(1, 11))}, 'sample': 'categorical'})
        }
        dropout_sig_dict = {
            'activation': Var(var_type='activation', source={'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]}, 'sample': 'categorical'}),
            'dropout_p': Var(var_type='dropout_p', source={'params': {'low': 0.0, 'high': 1.0}, 'sample': 'float'})
        }

        self.action_sampler = {
            action_default: Action(action_default, default_sig_dict),
            action_dropout: Action(action_dropout, dropout_sig_dict)
        }

        # Create a trial object for parameterization
        self.trial = optuna.create_study(direction='minimize').ask()

    def test_initialization(self):
        default_action = self.action_sampler[action_default]
        dropout_action = self.action_sampler[action_dropout]

        # Test if Action objects are initialized correctly
        self.assertIsInstance(default_action, Action)
        self.assertIsInstance(dropout_action, Action)

        self.assertTrue(default_action.action_func == action_default)
        self.assertIsInstance(default_action.get_var('rand_number'), Var)
        self.assertTrue(
            default_action.get_var('rand_number').distribution() == list(range(1, 11)),
            msg=f"{default_action.get_var('rand_number').distribution()} != {list(range(1, 11))}"
        )
        self.assertFalse(default_action.params)
        self.assertIsNone(default_action.action_state)
        self.assertIsNone(default_action._model_loader)

        self.assertTrue(dropout_action.action_func == action_dropout)
        self.assertIsInstance(dropout_action.get_var('activation'), Var)
        self.assertIsInstance(dropout_action.get_var('dropout_p'), Var)
        self.assertTrue(dropout_action.get_var('activation').distribution() == [nn.ReLU, nn.PReLU, nn.SiLU])
        self.assertFalse(dropout_action.params)
        self.assertIsNone(dropout_action.action_state)
        self.assertIsNone(dropout_action._model_loader)

    def test_default_parameterize_with_action_state(self):
        default_action = self.action_sampler[action_default]
        default_action.action_state = 'dummy'
        default_action.params = {'rand_number': 1}

        # Test parameterization with the default action
        default_action.parameterize(trial=self.trial)

        # Posterior testing
        self.assertIsInstance(default_action, Action)
        self.assertIsNotNone(default_action.params)
        self.assertIsNotNone(default_action.action_state)
        self.assertIsNotNone(default_action._model_loader)

        try:
            model = default_action(device=torch.device('cpu'))
        except Exception as e:
            pass

    def test_default_parameterize(self):
        default_action = self.action_sampler[action_default]
        # Test parameterization with the default action
        default_action.parameterize(trial=self.trial)
        self.assertTrue(set(default_action.params.keys()), set(['rand_number']))
        self.assertTrue(default_action.params['rand_number'] in list(range(1, 11)))
        self.assertIsNone(default_action.action_state)
        self.assertIsNotNone(default_action._model_loader)
        model = default_action(device=torch.device('cpu'))
        X = torch.randn(size=[10, 16])
        Y = model(X)
        self.assertEqual(Y.shape, torch.Size([10, 10]))

        self.assertIsNotNone(default_action.params)
        self.assertIsNone(default_action.action_state)
        self.assertIsNotNone(default_action._model_loader)

    def test_dropout_parameterize_with_action_state(self):
        dropout_action = self.action_sampler[action_dropout]
        dropout_action.params = {
            'activation': nn.ReLU,
            'dropout_p': 0.5
        }
        model = ModelDropout(activation=nn.ReLU, dropout_p=0.5).to(torch.device('cpu'))
        dropout_action.action_state = model.state_dict()

        # Test parameterization with the default action
        dropout_action.parameterize(trial=self.trial)

        # Posterior testing
        self.assertIsInstance(dropout_action, Action)
        self.assertIsNotNone(dropout_action.params)
        self.assertIsNotNone(dropout_action.action_state)
        self.assertIsNotNone(dropout_action._model_loader)

        try:
            model = dropout_action(device=torch.device('cpu'))
            self.assertTrue(model(torch.randn(size=[10, 16])).shape, torch.Size([10, 10]))
        except Exception as e:
            pass

        # Final testing
        self.assertIsNotNone(dropout_action.params)
        self.assertIsNotNone(dropout_action.action_state)
        self.assertIsNotNone(dropout_action._model_loader)

    def test_dropout_parameterize(self):
        dropout_action = self.action_sampler[action_dropout]

        # Prior Testing
        self.assertIsInstance(dropout_action, Action)
        self.assertTrue(dropout_action.params == {})
        self.assertIsNone(dropout_action.action_state)
        self.assertIsNone(dropout_action._model_loader)

        # Test parameterization with the default action
        dropout_action.parameterize(trial=self.trial)

        # Posterior Testing
        self.assertTrue(set(dropout_action.params.keys()), set(['activation', 'dropout_p']))
        self.assertTrue(dropout_action.params['activation'] in [nn.ReLU, nn.PReLU, nn.SiLU])
        self.assertTrue(0.0 <= dropout_action.params['dropout_p'] <= 1.0)
        self.assertIsNone(dropout_action.action_state)
        self.assertIsNotNone(dropout_action._model_loader)

        # Model loading & testing
        model = dropout_action(device=torch.device('cpu'))
        self.assertEqual(model(torch.randn(size=[10, 16])).shape, torch.Size([10, 10]))

        # Final testing
        self.assertIsNotNone(dropout_action.params)
        self.assertIsNone(dropout_action.action_state)
        self.assertIsNotNone(dropout_action._model_loader)

    def test_update_sig(self):
        # Preparation
        default_action = self.action_sampler[action_default]
        dropout_action = self.action_sampler[action_dropout]

        default_new_sig_params = {
            'rand_number': [2]
        }
        dropout_new_sig_params = {
            'activation': [1, 2],
            'dropout_p': [0.45, 0.5, 0.72]
        }

        # TEST: default_action FOR: sig_dict
        default_action.update_sig(update_params=default_new_sig_params)
        self.assertIsInstance(default_action.get_var('rand_number'), Var)
        self.assertTrue(default_action.get_var('rand_number').distribution() == [3])
        self.assertTrue(default_action.get_var('rand_number').distribution(indices=True) == [2])

        # TEST: dropout_action FOR: sig_dict
        dropout_action.update_sig(update_params=dropout_new_sig_params)
        self.assertIsInstance(dropout_action.get_var('activation'), Var)
        self.assertTrue(dropout_action.get_var('activation').distribution() == [nn.PReLU, nn.SiLU],
                        msg=f"{dropout_action.get_var('activation').distribution()} != {[nn.PReLU, nn.SiLU]}")
        self.assertTrue(dropout_action.get_var('activation').distribution(True) == [1, 2],
                        msg=f"{dropout_action.get_var('activation').distribution(True)} != {[1, 2]}")
        self.assertIsInstance(dropout_action.get_var('dropout_p'), Var)
        dropout_p_var = dropout_action.get_var('dropout_p')
        self.assertTrue(dropout_p_var.distribution()['low'] == 0.45 and dropout_p_var.distribution()['high'] == 0.72)

    # Additional method to test update_state if needed
    def test_update_state(self):
        for i in range(5):
            default_action = self.action_sampler[action_default]
            dropout_action = self.action_sampler[action_dropout]

            if not i:
                self.assertTrue(default_action.params == {})
                self.assertIsNone(default_action.action_state)
                self.assertIsNone(default_action._model_loader)

                self.assertTrue(dropout_action.params == {})
                self.assertIsNone(dropout_action.action_state)
                self.assertIsNone(dropout_action._model_loader)
            else:
                self.assertFalse(default_action.params == {})
                self.assertIsNotNone(default_action.action_state)
                self.assertIsNotNone(default_action._model_loader)

                self.assertFalse(dropout_action.params == {})
                self.assertIsNotNone(dropout_action.action_state)
                self.assertIsNotNone(dropout_action._model_loader)

            default_action.parameterize(trial=self.trial)
            dropout_action.parameterize(trial=self.trial)

            default_model = default_action(device=torch.device('cpu'))
            dropout_model = dropout_action(device=torch.device('cpu'))

            self.assertEqual(default_model(torch.randn(size=[10, 16])).shape, torch.Size([10, 10]))
            self.assertEqual(dropout_model(torch.randn(size=[10, 16])).shape, torch.Size([10, 10]))

            default_action.update_state(obj=default_model)
            dropout_action.update_state(obj=dropout_model)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

[I 2024-09-27 21:37:26,696] A new study created in memory with name: no-name-d3ffcc07-671d-4141-af90-0d7a1005a56f
.[I 2024-09-27 21:37:26,818] A new study created in memory with name: no-name-617f16d5-8123-4c44-b061-18f72dacb208
.[I 2024-09-27 21:37:26,830] A new study created in memory with name: no-name-14a98dd4-b6ed-4075-9a64-a476c0ede889
.[I 2024-09-27 21:37:26,885] A new study created in memory with name: no-name-98460ab6-a6f0-4411-85f6-c46979e51242
.[I 2024-09-27 21:37:26,916] A new study created in memory with name: no-name-32f78d07-55fa-46de-90db-286de227b1d9
.[I 2024-09-27 21:37:26,923] A new study created in memory with name: no-name-5ada5028-e7e0-41f8-8e7e-9c69cfafb7b7
.[I 2024-09-27 21:37:26,933] A new study created in memory with name: no-name-595af548-4d81-4b5f-a9c4-3ad02184030e
...........[I 2024-09-27 21:37:26,996] A new study created in memory with name: no-name-200a71fe-1cf9-4283-96e3-c50662573d03
..[I 2024-09-27 21:37:27,018] A new study created in memory with name: 

#Class: ActionSampler

In [6]:
class ActionSampler:
    def __init__(self, INIT_ACTION_DICT: dict):
        self.actions = dict()

        for action_func,sig_dict in INIT_ACTION_DICT.items():
            if not all(isinstance(var,Var) for var in sig_dict.values()):
                raise ValueError(f"Input dictionary for initializing object 'ActionSampler' with action_func: {action_func.__name__} has a non-Var value in it. Input Dictionary: {sig_dict}")
            self.actions[action_func] = Action(action_func=action_func, sig_dict=sig_dict)

    def __len__(self):
        return len(self.actions)

    def __eq__(self, other: ActionSampler):
        return isinstance(other, ActionSampler) and self.actions == other.actions

    def __str__(self):
        output = "**********ActionSampler**********\n"
        output += "".join([f"{action}\n" for action in self.actions.values()])
        return output

    def get_action(self, action_func):
        if action_func not in self.keys():
            raise KeyError(f"ActionSampler cannot retrieve Requested Action: {action_func.__name__ if callable(action_func) else action_func}, since it does NOT belong to: {self.keys()}")
        return self.actions[action_func]

    def keys(self):
        return self.actions.keys()

    def values(self):
        return self.actions.values()

    def items(self):
        return self.actions.items()

    def sample(self, action_func: Callable, trial: Trial):
        """
        SAMPLING THROUGH ActionSampler WILL ENFORCE THAT input 'trial'!=None.
        (Much like the TrainSampler)
        """
        if trial is not None and not isinstance(trial, (Trial, FrozenTrial)):
            raise ValueError(f"Input 'trial' should be an object optuna::Trial or optuna::FrozenTrial but found: {trial}")
        if trial is None:
            raise ValueError(f"Input 'trial' cannot be None when sampling is requested through ActionSampler. Either pass in as trial an active TrialOR manually build an Action function")

        #action <- Action object that encapsulates 'action_func'
        action = self.get_action(action_func)

        #Parameterize 'action'.
        action.parameterize(trial=trial)

        #RETURN: Fully parameterized 'action'.
        #NOTE: 'action' could be either be sampled from 'trial' or loaded from saved signatures(if 'action' has a saved action_state or trial==None)
        return action

    def get_loadable_action(self, action_func: Callable, source):
        """
        ASSUMPTION: source is a dictionary of parameters whose categorical values are all indices.
        """
        if not isinstance(source, (Trial, FrozenTrial, dict)):
            raise ValueError(f"Input 'source' must be of type in (optuna::Trial, optuna::FrozenTrial, dict) but found: {source}")

        action = self.get_action(action_func)
        #IF action has a saved 'action_state', THEN return action object.
        if action.action_state:
            return action

        #source <- source.params if isinstance(Trial, FrozenTrial)
        source = source if isinstance(source, dict) else source.params
        #leftovers <- action.keys() that are not covered by source.keys()
        leftovers = set(action.keys()) - set(source.keys())
        if leftovers:
            raise ValueError(f"Action: {action_func.__name__} requires following missing keys in 'source': {leftovers}")

        #Recast the values of 'source' using action.items()
        source = {key:var.convert_idx_to_val(idx=source[key]) for key,var in action.items()}

        #OVERRIDE: action._model_loader
        action._model_loader = partial(action_func, **source)

        return action

    def get_action_to_profile(self, list_params: list):
        """
        TO BE CALLED IN Sampler.get_default_dict() AFTER .get_evaluated_params()
        list_of_params := list of params produced by .get_evaluated_params() on each trial of top_k_trials.
        """
        action_to_profile = defaultdict(lambda: defaultdict(list))
        profiled_actions = set()
        action_funcs = self.keys()

        for params in list_params:
            if 'action_func' not in params.keys(): continue
            #action_func <- Callable 'action_func'
            action_func = params['action_func']
            #Sanity Check1:
            assert action_func in action_funcs
            #action <- Action object that reprs 'action_func'
            action = self.get_action(action_func)
            #Sanity Check2:
            assert isinstance(action, Action)
            #action_state <- state dictionary of the 'action' object.
            action_state = action.action_state
            params_sig_keys = set(params.keys() & action.keys())
            params_train_keys = set(params.keys()) - params_sig_keys

            #ERROR CHECKS:
            if action_state and params_sig_keys:
                raise ValueError(f"Action: {action.action_func.__name__} has a saved action_state but 'params' has the following keys: {params_sig_keys}")
            if not action_state and params_sig_keys!=set(action.keys()):
                raise ValueError(f"Action: {action.action_func.__name__} does NOT have a saved action_state but params_sig_keys: {params_sig_keys} != action.keys(): {action.keys()}")

            #profile <- Dict of lists according to the 'action_func'
            profile = action_to_profile[action_func]
            profile.update({key:profile[key] + [val] for key,val in params.items() if key in params_sig_keys})
            if action_func not in profiled_actions:
                profiled_actions.add(action_func)
                profile.update({key:profile[key] + [val] for key,val in params.items() if key in params_train_keys})

        return action_to_profile

    def update_vars(self, action_to_profile: dict):
        for action_func,profile in action_to_profile.items():
            self.get_action(action_func).update_sig(update_params=profile)

    def update_states(self, action_to_profile: dict, model_train_window: int, device: torch.device):
        m = model_train_window

        for action_func, profile in list(action_to_profile.items())[:m]:
            #default_dict <- only the best value(s) for each (signature)parameter extracted.
            default_dict = {var_type:vals[0] for var_type,vals in profile.items()}
            #action <- loadable action created with best signature parameters
            action = self.get_loadable_action(
                action_func=action_func,
                source=default_dict
            )

            assert action.is_loadable(), f"action: {action}"

            #UPDATE: 'default_dict' with ('action_func'->manual_action)
            default_dict.update({'action_func':action})

            trainer, learner = Wrapper(
                trial=None,
                sampler=None,
                device=device,
                default_dict=default_dict
            ).yield_execution_set()
            trainer()
            action.update_state(obj=learner)

#Test: ActionSampler

In [10]:
class ModelDefault(nn.Module):
    def __init__(self, rand_number: int):
        super().__init__()
        self.net = nn.Linear(16,10)
        self.rand_number = torch.tensor(rand_number)

    def forward(self, x):
        return self.net(x) + self.rand_number

class ModelDropout(nn.Module):
    def __init__(self, activation: nn.Module, dropout_p: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features=16,out_features=10),
            activation(),
            nn.Dropout(p=dropout_p),
            nn.Linear(in_features=10,out_features=10)
        )

    def forward(self, x):
        return self.net(x)

def action_default(rand_number: int, device: torch.device):
    return ModelDefault(rand_number=rand_number).to(device)

def action_dropout(activation, dropout_p, device: torch.device):
    #model_dropout := Class header of 'ModelDropout'
    return ModelDropout(activation=activation, dropout_p=dropout_p).to(device)

#Wrapper <- Mock object
#NOTE: Details of this object will be filled in the next cell(inside test(s) of 'TestActionSampler')
Wrapper = Mock()

In [11]:
class TestActionSampler(unittest.TestCase):
    def setUp(self):
        self.INIT_DICT = {
            'rand_number': Var(var_type='rand_number', source={'params': {'choices': list(range(1, 11))}, 'sample': 'categorical'}),
            'activation': Var(var_type='activation', source={'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]}, 'sample': 'categorical'}),
            'dropout_p': Var(var_type='dropout_p', source={'params': {'low': 0.0, 'high': 1.0}, 'sample': 'float'})
        }

        action_to_sig = {func: set(signature(func).parameters.keys()) - {'device'} for func in [action_default, action_dropout]}
        self.INIT_ACTION_DICT = {func: {key: self.INIT_DICT[key] for key in sig} for func, sig in action_to_sig.items()}
        self.trial = optuna.create_study(direction='minimize').ask()

    def test_initialization(self):
        action_sampler = ActionSampler(INIT_ACTION_DICT=self.INIT_ACTION_DICT)
        self.assertEqual(len(action_sampler.actions), 2)
        self.assertIn(action_default, action_sampler.actions)
        self.assertIn(action_dropout, action_sampler.actions)
        self.assertTrue(all(isinstance(action, Action) for action in action_sampler.actions.values()))

        default_sig_dict = {'rand_number': Var(var_type='rand_number', source={'params': {'choices': list(range(1, 11))}, 'sample': 'categorical'})}
        dropout_sig_dict = {
            'activation': Var(var_type='activation', source={'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]}, 'sample': 'categorical'}),
            'dropout_p': Var(var_type='dropout_p', source={'params': {'low': 0.0, 'high': 1.0}, 'sample': 'float'})
        }

        self.assertTrue(action_sampler.get_action(action_default) == Action(action_default, default_sig_dict))
        self.assertTrue(action_sampler.get_action(action_dropout) == Action(action_dropout, dropout_sig_dict))

    def test_sample_action(self):
        action_sampler = ActionSampler(INIT_ACTION_DICT=self.INIT_ACTION_DICT)
        for func in [action_default, action_dropout]:
            sampled_action = action_sampler.sample(
                action_func=func,
                trial=optuna.create_study(direction='minimize').ask()
            )
            self.assertIsInstance(sampled_action, Action)
            expected_keys = ['rand_number'] if func == action_default else ['activation', 'dropout_p']
            self.assertTrue(set(sampled_action.params.keys()) == set(expected_keys))
            self.assertTrue(set(sampled_action._model_loader.keywords.keys()) == set(expected_keys))
            self.assertIsNone(sampled_action.action_state)
            model = sampled_action(device=torch.device('cpu'))
            self.assertIsInstance(model, nn.Module)
            self.assertEqual(model(torch.randn(size=[10, 16])).shape, torch.Size([10, 10]))

    def test_get_loadable_action_with_dict(self):
        action_sampler = ActionSampler(INIT_ACTION_DICT=self.INIT_ACTION_DICT)
        for func in [action_default, action_dropout]:
            loadable_action = action_sampler.get_loadable_action(
                action_func=func,
                source={'rand_number': 1} if func == action_default else {'activation': 0, 'dropout_p': 0.5}
            )
            self.assertIsInstance(loadable_action, Action)
            self.assertTrue(
                set(loadable_action._model_loader.keywords.keys()) == set(['rand_number']) if func == action_default else set(['activation', 'dropout_p']),
                msg=f"loadable_action._model_loader.keywords: {loadable_action._model_loader.keywords}"
            )
            self.assertIsNone(loadable_action.action_state)
            model = loadable_action(device=torch.device('cpu'))
            self.assertIsInstance(model, nn.Module)
            self.assertEqual(model(torch.randn(size=[10, 16])).shape, torch.Size([10, 10]))

    def test_get_loadable_action_with_trial(self):
        action_sampler = ActionSampler(INIT_ACTION_DICT=self.INIT_ACTION_DICT)
        for func in [action_default, action_dropout]:
            trial = optuna.create_study(direction='minimize').ask()
            dummy = action_sampler.sample(action_func=func, trial=trial)  # Sample to simulate state changes.
            loadable_action = action_sampler.get_loadable_action(
                action_func=func,
                source=trial
            )
            self.assertIsInstance(loadable_action, Action)
            expected_keys = ['rand_number'] if func == action_default else ['activation', 'dropout_p']
            self.assertTrue(set(loadable_action.params.keys()) == set(expected_keys),
                            msg=f"loadable_action.params: {loadable_action.params}")
            self.assertTrue(set(loadable_action._model_loader.keywords.keys()) == set(expected_keys),
                            msg=f"loadable_action._model_loader.keywords: {loadable_action._model_loader.keywords}")
            self.assertIsNone(loadable_action.action_state)
            model = loadable_action(device=torch.device('cpu'))
            self.assertIsInstance(model, nn.Module)
            self.assertEqual(model(torch.randn(size=[10, 16])).shape, torch.Size([10, 10]))

    def test_sample_with_invalid_trial(self):
        action_sampler = ActionSampler(INIT_ACTION_DICT=self.INIT_ACTION_DICT)
        with self.assertRaises(ValueError):
            action_sampler.sample(action_default, None)
        with self.assertRaises(ValueError):
            action_sampler.sample(action_dropout, None)

        # Test sampling with invalid trial type
        with self.assertRaises(ValueError):
            action_sampler.sample(action_default, "invalid_trial")
        with self.assertRaises(ValueError):
            action_sampler.sample(action_dropout, "invalid_trial")

    def test_get_action_to_profile(self):
        list_params = [
            {'action_func': action_dropout, 'activation': 1, 'dropout_p': 0.78, 'lr': 1e-1, 'n_epoch': 10},
            {'action_func': action_default, 'rand_number': 1, 'lr': 1e-2, 'n_epoch': 12},
            {'action_func': action_dropout, 'activation': 2, 'dropout_p': 0.123, 'lr': 1e-3, 'n_epoch': 14},
            {'action_func': action_default, 'rand_number': 1, 'lr': 1e-9, 'n_epoch': 20},
            {'action_func': action_default, 'rand_number': 3, 'lr': 2e-3, 'n_epoch': 1}
        ]

        action_sampler = ActionSampler(INIT_ACTION_DICT=self.INIT_ACTION_DICT)
        action_to_profile = action_sampler.get_action_to_profile(list_params=list_params)

        for action_func, profile in action_to_profile.items():
            self.assertIn(action_func, action_sampler.keys())

            if action_func == action_default:
                self.assertEqual(profile['rand_number'], [1, 1, 3], msg=f"profile['rand_number']: {profile['rand_number']}")
                self.assertEqual(profile['lr'], [1e-2], msg=f"profile['lr']: {profile['lr']}")
                self.assertEqual(profile['n_epoch'], [12], msg=f"profile['n_epoch']: {profile['n_epoch']}")
            else:
                self.assertEqual(profile['dropout_p'], [0.78, 0.123], msg=f"profile['dropout_p']: {profile['dropout_p']}")
                self.assertEqual(profile['activation'], [1, 2], msg=f"profile['activation']: {profile['activation']}")
                self.assertEqual(profile['lr'], [1e-1], msg=f"profile['lr']: {profile['lr']}")
                self.assertEqual(profile['n_epoch'], [10], msg=f"profile['n_epoch']: {profile['n_epoch']}")

    def test_update_vars(self):
        list_params = [
            {'action_func': action_dropout, 'activation': 1, 'dropout_p': 0.78, 'lr': 1e-1, 'n_epoch': 10},
            {'action_func': action_default, 'rand_number': 1, 'lr': 1e-2, 'n_epoch': 12},
            {'action_func': action_dropout, 'activation': 2, 'dropout_p': 0.123, 'lr': 1e-3, 'n_epoch': 14},
            {'action_func': action_default, 'rand_number': 1, 'lr': 1e-9, 'n_epoch': 20},
            {'action_func': action_default, 'rand_number': 3, 'lr': 2e-3, 'n_epoch': 1}
        ]
        action_sampler = ActionSampler(INIT_ACTION_DICT=self.INIT_ACTION_DICT)
        action_to_profile = action_sampler.get_action_to_profile(list_params=list_params)
        action_sampler.update_vars(action_to_profile)

        # Check the updated variables for action_default
        default_action = action_sampler.get_action(action_default)
        rand_var = Var(var_type='rand_number', source={'params': {'choices': list(range(1, 11))}, 'sample': 'categorical'})
        rand_var._dist = [1, 1, 3]
        self.assertEqual(default_action.sig_dict, {'rand_number': rand_var}, msg=f"default_action.sig_dict: {default_action.sig_dict}")

        # Check the updated variables for action_dropout
        dropout_action = action_sampler.get_action(action_dropout)
        activation_var = Var(var_type='activation', source={'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]}, 'sample': 'categorical'})
        activation_var._dist = [1, 2]
        dropout_p_var = Var(var_type='dropout_p', source={'params': {'low': 0.0, 'high': 1.0}, 'sample': 'float'})
        dropout_p_var._dist = {'low': 0.123, 'high': 0.78}
        self.assertTrue(dropout_action.get_var('activation') == activation_var,
                        msg=f"dropout_action['activation']: {dropout_action.get_var('activation')} != \nactivation_var: {activation_var}")
        self.assertTrue(dropout_action.get_var('dropout_p') == dropout_p_var,
                        msg=f"dropout_action['dropout_p']: {dropout_action.get_var('dropout_p')} != \ndropout_p_var: {dropout_p_var}")

    def mock_wrapper_init(self, trial, default_dict, sampler, device):
        action = default_dict['action_func']
        self.assertIsInstance(action, Action)
        self.assertIsNotNone(action.params)
        self.assertIsNone(action.action_state)
        self.assertIsNotNone(action._model_loader)

        # Create the model after checking action is valid
        model = action(device=device)
        self.assertIsInstance(model, nn.Module)
        self.assertEqual(model(torch.randn(size=[10, 16])).shape, torch.Size([10, 10]))

        trainer = Mock()
        trainer.side_effect = lambda: [param.data.fill_(0) for param in model.parameters()]

        execution_set = Mock()
        execution_set.yield_execution_set.return_value = (trainer, model)

        return execution_set

    def produce_saved_bools(self, action_sampler: ActionSampler):
        dropout_action = action_sampler.get_action(action_dropout)
        default_action = action_sampler.get_action(action_default)
        default_saved = default_action.action_state is not None
        dropout_saved = dropout_action.action_state is not None

        if default_saved:
            for param_tensor in default_action.action_state.values():
                if param_tensor.numel() > 0:  # Check tensor is not empty
                    default_saved &= torch.all(param_tensor == 0).item()
        if dropout_saved:
            for param_tensor in dropout_action.action_state.values():
                if param_tensor.numel() > 0:  # Check tensor is not empty
                    dropout_saved &= torch.all(param_tensor == 0).item()
        return default_saved, dropout_saved

    def test_update_states(self):
        param_sets = [
            {
                'params': [
                    {'action_func': action_dropout, 'activation': 1, 'dropout_p': 0.78, 'lr': 1e-1, 'n_epoch': 10},
                    {'action_func': action_default, 'rand_number': 1, 'lr': 1e-2, 'n_epoch': 12},
                    {'action_func': action_dropout, 'activation': 2, 'dropout_p': 0.123, 'lr': 1e-3, 'n_epoch': 14},
                    {'action_func': action_default, 'rand_number': 1, 'lr': 1e-9, 'n_epoch': 20},
                    {'action_func': action_default, 'rand_number': 3, 'lr': 2e-3, 'n_epoch': 1},
                ],
                'model_train_window': 1,
                'xy': (False, True)
            },
            {
                'params': [
                    {'action_func': action_default, 'rand_number': 1, 'lr': 1e-2, 'n_epoch': 12},
                    {'action_func': action_dropout, 'activation': 1, 'dropout_p': 0.78, 'lr': 1e-1, 'n_epoch': 10},
                    {'action_func': action_dropout, 'activation': 2, 'dropout_p': 0.123, 'lr': 1e-3, 'n_epoch': 14},
                    {'action_func': action_default, 'rand_number': 1, 'lr': 1e-9, 'n_epoch': 20},
                    {'action_func': action_default, 'rand_number': 3, 'lr': 2e-3, 'n_epoch': 1},
                ],
                'model_train_window': 1,
                'xy': (True, False)
            },
            {
                'params': [
                    {'action_func': action_dropout, 'activation': 1, 'dropout_p': 0.78, 'lr': 1e-1, 'n_epoch': 10},
                    {'action_func': action_default, 'rand_number': 1, 'lr': 1e-2, 'n_epoch': 12},
                    {'action_func': action_dropout, 'activation': 2, 'dropout_p': 0.123, 'lr': 1e-3, 'n_epoch': 14},
                    {'action_func': action_default, 'rand_number': 1, 'lr': 1e-9, 'n_epoch': 20},
                    {'action_func': action_default, 'rand_number': 3, 'lr': 2e-3, 'n_epoch': 1},
                ],
                'model_train_window': 2,
                'xy': (True, True)
            },
            {
                'params': [
                    {'action_func': action_dropout, 'activation': 1, 'dropout_p': 0.78, 'lr': 1e-1, 'n_epoch': 10},
                    {'action_func': action_dropout, 'activation': 2, 'dropout_p': 0.123, 'lr': 1e-3, 'n_epoch': 14},
                    {'action_func': action_default, 'rand_number': 1, 'lr': 1e-9, 'n_epoch': 20}
                ],
                'model_train_window': 2,
                'xy': (True, True)
            },
            {
                'params': [
                    {'action_func': action_dropout, 'activation': 1, 'dropout_p': 0.78, 'lr': 1e-1, 'n_epoch': 10},
                    {'action_func': action_dropout, 'activation': 2, 'dropout_p': 0.123, 'lr': 1e-3, 'n_epoch': 14},
                    {'action_func': action_default, 'rand_number': 1, 'lr': 1e-9, 'n_epoch': 20}
                ],
                'model_train_window': 0,
                'xy': (False, False)
            }
        ]

        for params in param_sets:
            action_sampler = ActionSampler(INIT_ACTION_DICT=self.INIT_ACTION_DICT)
            Wrapper.side_effect = self.mock_wrapper_init
            list_params, model_train_window, xy_label = params['params'], params['model_train_window'], params['xy']

            action_to_profile = action_sampler.get_action_to_profile(list_params=list_params)
            action_sampler.update_vars(action_to_profile)
            action_sampler.update_states(action_to_profile, model_train_window, torch.device('cpu'))

            xy = self.produce_saved_bools(action_sampler)
            self.assertEqual(xy, xy_label)


if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

[I 2024-09-27 21:37:27,720] A new study created in memory with name: no-name-9057a1a6-28a0-49ab-bf6e-8f49762f06e2
.[I 2024-09-27 21:37:27,735] A new study created in memory with name: no-name-af36177c-776c-4d2e-bf17-a0e5e30f0cb9
.[I 2024-09-27 21:37:27,745] A new study created in memory with name: no-name-14237fc2-b422-4771-87ea-50238f9fd62f
.[I 2024-09-27 21:37:27,774] A new study created in memory with name: no-name-1b8d6390-2e40-49a2-90a8-7797d38980bf
.[I 2024-09-27 21:37:27,790] A new study created in memory with name: no-name-6ed84615-5784-49da-8f76-de3a94bad625
.[I 2024-09-27 21:37:27,810] A new study created in memory with name: no-name-0a63d515-9810-4304-a002-b6ce1140bf9e
.[I 2024-09-27 21:37:27,819] A new study created in memory with name: no-name-e725afb9-03e6-465a-9e8b-bb70fb9aaa30
.[I 2024-09-27 21:37:27,872] A new study created in memory with name: no-name-a372b31a-0d9d-4e45-9703-4fbf1e440a45
.[I 2024-09-27 21:37:27,887] A new study created in memory with name: no-name-165

#Class: TrainSampler

In [7]:
class TrainSampler:
    def __init__(self, INIT_TRAIN_DICT: dict):
        if not all(isinstance(var,Var) for var in INIT_TRAIN_DICT.values()):
            raise ValueError(f"Input dictionary for initializing object 'TrainSampler' has a non-Var value in it. Input dictionary: {INIT_TRAIN_DICT}")
        self.sample_dict = {cat:val for cat,val in INIT_TRAIN_DICT.items()}

    def __len__(self):
        return len(self.sample_dict)

    def __eq__(self, other: TrainSampler):
        return isinstance(other, TrainSampler) and self.sample_dict == other.sample_dict

    def __str__(self):
        output = "**********TrainSampler**********\n"
        output += "".join([f"{var}\n" for var in self.sample_dict.values()])
        return output

    def get_var(self, var_type):
        if var_type not in self.keys():
            raise KeyError(f"Requested 'var_type': {var_type} is not in train keys: {self.keys()}.")
        return self.sample_dict[var_type]

    def keys(self):
        return self.sample_dict.keys()

    def values(self):
        return self.sample_dict.values()

    def items(self):
        return self.sample_dict.items()

    def is_fixed(self):
        return all(var.fixed for var in self.values())

    def get_evaluated_params(self, source):
        if not isinstance(source, (Trial, FrozenTrial, dict)):
            raise ValueError(f"Input 'source' must be of type in (optuna::Trial, optuna::FrozenTrial, dict) but found: {source}")

        source = source if isinstance(source, dict) else source.params

        #return source | {var_type:self.get_var(var_type).convert_idx_to_val(source[var_type]) for var_type in self.keys()&source.keys()}
        return source | {var_type:self.get_var(var_type).convert_idx_to_val(idx) for var_type,idx in source.items() if var_type in self.keys()}

    def sample(self, var_type: str, trial):
        if trial is not None and not isinstance(trial, (Trial, FrozenTrial)):
            raise ValueError(f"Input 'trial' should be an instance of optuna::Trial OR optuna::FrozenTrial but found: {trial}")
        if trial is None:
            raise ValueError("Input 'trial' CANNOT be 'None'. Make sure to throw in optuna::Trial OR optuna::FrozenTrial")
        if var_type not in self.keys():
            raise ValueError(f"Requested var_type: {var_type} does NOT exist in the called TrainSampler!")

        #sampled_val <- value (of category 'var_type') sampled from 'sample_dict' with trial 'trial'.
        sampled_val = self.get_var(var_type).sample(trial=trial)

        return sampled_val

    def update_vars(self, top_k_trials: list):
        #Acc over the list of trials for each parameter key.
        for key,var in self.items():
            var.update(vals=[trial.params[key] for trial in top_k_trials if key in trial.params.keys()])

#Class: Sampler

In [8]:
class Sampler:
    def __init__(self, INIT_DICT: dict):
        #Preprocessing:
        init_keys = set(INIT_DICT.keys())
        self._sig_keys = set()
        INIT_ACTION_DICT = dict()

        for action_func in set(INIT_DICT['action_func']['params']['choices']):
            sig = set(signature(action_func).parameters.keys()) - {'device'}
            leftovers = sig - init_keys
            if leftovers:
                raise ValueError(f"Missing keys for parameterizing '{action_func.__name__}': [{leftovers}]. Required keys: {sig}, provided keys: {init_keys}.")
            #INIT_ACTION_DICT[action_func] <- Dictionary of mappings (var_type->Var ) for every var_type in 'sig'.
            INIT_ACTION_DICT[action_func] = {var_type:Var(var_type=var_type,source=INIT_DICT[var_type]) for var_type in sig}
            #self.sig_keys <- Union(self.action_keys, sig)
            self._sig_keys |= sig

        #INIT_TRAIN_DICT <- {(var_type->Var )|var_type NOT IN action_keys}
        INIT_TRAIN_DICT = {var_type:Var(var_type=var_type,source=INIT_DICT[var_type]) for var_type in init_keys-self._sig_keys}

        #train_keys <- all the keys that belong to the TrainSampler's space.
        train_keys = set(INIT_TRAIN_DICT.keys())

        #Make sure that 'action_func' is in 'INIT_DICT_TRAIN'.
        if 'action_func' not in train_keys:
            raise ValueError("Reconstructed 'INIT_TRAIN_DICT' must have 'action_func' as a valid key but not found.")

        #Declare two Samplers, after pre-processing is done.
        self.action_sampler = ActionSampler(INIT_ACTION_DICT=INIT_ACTION_DICT)
        self.train_sampler = TrainSampler(INIT_TRAIN_DICT=INIT_TRAIN_DICT)
        #Fix the hidden_keys
        self._sig_keys = frozenset(self._sig_keys)

    def __str__(self):
        output = "Sampler:\n"
        output += f"{self.action_sampler}\n"
        return output + f"{self.train_sampler}\n"

    def get_action(self, action_func) -> Action:
        action_func_var = self.get_var('action_func')
        action_func = action_func_var.convert_idx_to_val(action_func) if isinstance(action_func, int) else action_func
        return self.action_sampler.get_action(action_func)

    def get_var(self, var_type, sig_var_type=None) -> Var:
        #sig_var_type is None <-> var_type in self.train_sampler.keys()
        if sig_var_type is None:
            return self.train_sampler.get_var(var_type)

        action = self.get_action(action_func=var_type)  # Get the associated action
        return action.get_var(var_type=sig_var_type)

    ###Inherited Methods:
    def convert_idx_to_val(self, var_type, sig_var_type_or_idx, idx=None):
        if idx is None:
            return self.get_var(var_type).convert_idx_to_val(idx=sig_var_type_or_idx)
        return self.get_var(var_type, sig_var_type_or_idx).convert_idx_to_val(idx=idx)

    def convert_val_to_idx(self, var_type, sig_var_type_or_val, val=None):
        if val is None:
            return self.get_var(var_type).convert_val_to_idx(val=sig_var_type_or_val)
        return self.get_var(var_type, sig_var_type_or_val).convert_val_to_idx(val=val)

    def contain_idx(self, var_type, sig_var_type_or_idx, idx=None):
        if idx is None:
            return self.get_var(var_type).contain_idx(sig_var_type_or_idx)
        return self.get_var(var_type, sig_var_type_or_idx).contain_idx(idx)

    def contain_val(self, var_type, sig_var_type_or_val, val=None):
        if val is None:
            return self.get_var(var_type).contain_val(sig_var_type_or_val)
        return self.get_var(var_type,sig_var_type_or_val).contain_val(val)
    ###Inherited Methods:

    def sig_keys(self):
        return self._sig_keys

    def train_keys(self):
        return self.train_sampler.keys()

    def action_funcs(self):
        return self.action_sampler.keys()

    def is_fixed(self):
        action_func = self.get_var('action_func').distribution()[0]
        action = self.get_action(action_func)
        return self.train_sampler.is_fixed() and action.is_fixed()

    def sample(self, var_type: str, trial: Trial, default_val = None):
        """
        For the sampling to be done properly, the trial == optuna::Trial
        (IF trial==None, THEN default_val will be returned)
        """
        train_keys = self.train_keys()
        action_funcs = self.action_funcs()
        var = self.get_var(var_type)
        valid_trial = trial is not None and isinstance(trial, (Trial, FrozenTrial))
        #NOTE: Any 'default_val' that gets passed into Sampler.sample MUST be in its current distribution.
        if not valid_trial and not var.contain_val(default_val):
            raise ValueError(f"Var of type: {var_type} CANNOT produce the given 'default_val': {default_val} since its current distribution is: {var.distribution()}")
        if not valid_trial:
            #IF NOT valid_trial YET 'default_val' IS in the Var:var_type's distribution,
            #THEN return 'default_val'
            return default_val

        #sampled_val <- sampled value of type: var_type from: self.train_sampler
        sampled_val = self.train_sampler.sample(var_type=var_type, trial=trial)
        #IF 'sampled_val' is NOT one of the action_func(s), THEN return the sampled result.
        if sampled_val not in action_funcs:
            return sampled_val
        #ELSE, sample from internal action sampler. This will return a fully parameterized Action object.
        return self.action_sampler.sample(action_func=sampled_val, trial=trial)

    def get_action_to_profile(self, top_k_trials: list):
        list_params = [self.train_sampler.get_evaluated_params(source=trial) for trial in top_k_trials]
        action_to_profile = self.action_sampler.get_action_to_profile(list_params=list_params)
        return action_to_profile

    def update(self, top_k_trials: list, model_train_window: int, device: torch.device):
        #UPDATE: train_sampler
        self.train_sampler.update_vars(top_k_trials=top_k_trials)

        #PRODUCE: action_to_profile
        #UPDATE: action_sampler Var(s) WITH: action_to_profile
        action_to_profile = self.get_action_to_profile(top_k_trials=top_k_trials)
        self.action_sampler.update_vars(action_to_profile=action_to_profile)

        #UPDATE: action_sampler WITH: action_to_profile
        self.action_sampler.update_states(
            action_to_profile=action_to_profile,
            model_train_window=min(model_train_window, len(action_to_profile)),
            device=device
        )

#Test: Sampler



In [14]:
class ModelDefault(nn.Module):
    def __init__(self, rand_number: int):
        super().__init__()
        self.net = nn.Linear(16,10)
        self.rand_number = torch.tensor(rand_number)

    def forward(self, x):
        return self.net(x) + self.rand_number

class ModelDropout(nn.Module):
    def __init__(self, activation: nn.Module, dropout_p: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features=16,out_features=10),
            activation(),
            nn.Dropout(p=dropout_p),
            nn.Linear(in_features=10,out_features=10)
        )

    def forward(self, x):
        return self.net(x)

class ModelDropoutVariant(nn.Module):
    """
    Same parameters as ModelDropout. This model will be used in the test
    to check the independence of signature Var(s).
    i.e. IF different values were thrown into .update() of ModelDropout VS ModelDropoutVariant,
         THEN their signature Var(s), though they have the same var_type, ._frozen_dist, they WILL BE TREATED AS DIFFERENT/INDEPENDENT objects.
    """
    def __init__(self, activation: nn.Module, dropout_p: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features=16,out_features=10),
            activation(),
            nn.Dropout(p=dropout_p),
            nn.Linear(in_features=10,out_features=10)
        )

    def forward(self, x):
        return self.net(x)

def action_default(rand_number: int, device: torch.device):
    return ModelDefault(rand_number=rand_number).to(device)

def action_dropout(activation, dropout_p, device: torch.device):
    #model_dropout := Class header of 'ModelDropout'
    return ModelDropout(activation=activation, dropout_p=dropout_p).to(device)

def action_dropout_variant(activation, dropout_p, device: torch.device):
    #model_dropout := Class header of 'ModelDropout'
    return ModelDropout(activation=activation, dropout_p=dropout_p).to(device)

#To be used in the next cell, to imitate the list of frozen optuna.Trial(s).
#optimized_study = Mock()

In [15]:
class TestSampler(unittest.TestCase):
    def setUp(self):
        self.INIT_DICT = {
            'action_func': {
                'params': {
                    'choices': [action_default, action_dropout, action_dropout_variant]
                },
                'sample': 'categorical'
            },
            'rand_number': {
                'params': {
                    'choices': list(range(1, 11))
                },
                'sample': 'categorical'
            },
            'activation': {
                'params': {
                    'choices': [nn.ReLU, nn.PReLU, nn.SiLU]
                },
                'sample': 'categorical'
            },
            'dropout_p': {
                'params': {
                    'low': 0.0,
                    'high': 1.0
                },
                'sample': 'float'
            },
            'lr': {
                'params': {
                    'low': 1e-4,
                    'high': 1e-2,
                    'log': True
                },
                'sample': 'float'
            },
            'wd': {
                'params': {
                    'low': 1e-4,
                    'high': 1e-1,
                    'log': True
                },
                'sample': 'float'
            },
            'batch_size': {
                'params': {
                    'choices': [32, 64, 128, 256]
                },
                'sample': 'categorical'
            },
            'n_epoch': {
                'params': {
                    'choices': [5, 10, 15]
                },
                'sample': 'categorical'
            }
        }
        self.trial = optuna.create_study(direction='minimize').ask()

    def test_initialization(self):
        sampler = Sampler(INIT_DICT=self.INIT_DICT)

        # Check keys and contents after initialization
        self.assertEqual(sampler.sig_keys(), frozenset(['rand_number', 'activation', 'dropout_p']))
        self.assertEqual(sampler.action_funcs(), set([action_default, action_dropout, action_dropout_variant]))
        self.assertEqual(sampler.train_keys(), set(['action_func', 'lr', 'wd', 'batch_size', 'n_epoch']))

        #Check action sampler
        expected_action_sampler = self.make_action_sampler(
            rand_choices=list(range(1, 11)),
            activation_choices=[nn.ReLU, nn.PReLU, nn.SiLU],
            dropout_tuple=(0.0, 1.0)
        )
        self.assertEqual(sampler.action_sampler, expected_action_sampler)

        #Check train sampler
        expected_train_sampler = self.make_train_sampler(
            action_func_choices=[action_default, action_dropout, action_dropout_variant],
            lr_tuple=(1e-4, 1e-2),
            wd_tuple=(1e-4, 1e-1),
            batch_size_choices=[32, 64, 128, 256],
            n_epoch_choices=[5, 10, 15]
        )
        self.assertEqual(sampler.train_sampler, expected_train_sampler)

    def test_sample(self):
        sampler = Sampler(INIT_DICT=self.INIT_DICT)
        self.assertTrue(sampler.sig_keys() == frozenset(['rand_number', 'activation', 'dropout_p']))
        self.assertTrue(sampler.action_funcs() == set([action_default, action_dropout, action_dropout_variant]))
        self.assertTrue(sampler.train_keys() == set(['action_func', 'lr', 'wd', 'batch_size', 'n_epoch']))

        for key in sampler.sig_keys():
            with self.assertRaises(KeyError):
                sampler.sample(var_type=key, trial=self.trial, default_val=None)

        for key in sampler.train_keys():
            with self.assertRaises(ValueError):
                #ValueError due to 'default_val' NOT belonging to the distribution of each var_type
                sampler.sample(var_type=key, trial=None, default_val=None)
            var = sampler.get_var(key)
            self.assertIsInstance(var, Var)
            self.assertTrue(var.var_type==key)
            dist = var.distribution()
            default_val = dist['low'] if var.sample_method!='categorical' else random.choices(dist)[0]
            self.assertTrue(var.contain_val(default_val),
                            msg=f"default_val: {default_val} NOT in distribution: {dist} of Var: {var.var_type}")
            sampled_val = sampler.sample(var_type=key, trial=None, default_val=default_val)
            self.assertEqual(sampled_val, default_val)

        for key in sampler.train_keys():
            sampled_val = sampler.sample(var_type=key, trial=self.trial, default_val=None)
            var = sampler.get_var(key)
            self.assertTrue(var.contain_val(sampled_val),
                            msg=f"sampled_val: {sampled_val} NOT in distribution: {var.distribution()}")

    def make_action_sampler(self, rand_choices, activation_choices, dropout_tuple):
        INIT_ACTION_DICT = {
            'rand_number': Var(var_type='rand_number', source={'params': {'choices': rand_choices}, 'sample': 'categorical'}),
            'activation': Var(var_type='activation', source={'params': {'choices': activation_choices}, 'sample': 'categorical'}),
            'dropout_p': Var(var_type='dropout_p', source={'params': {'low': dropout_tuple[0], 'high': dropout_tuple[1]}, 'sample': 'float'})
        }
        action_to_sig = {func: set(signature(func).parameters.keys()) - {'device'} for func in [action_default, action_dropout, action_dropout_variant]}
        INIT_ACTION_DICT = {func: {key: INIT_ACTION_DICT[key] for key in sig} for func, sig in action_to_sig.items()}
        return ActionSampler(INIT_ACTION_DICT=INIT_ACTION_DICT)

    def make_train_sampler(self, action_func_choices, lr_tuple, wd_tuple, batch_size_choices, n_epoch_choices):
        INIT_TRAIN_DICT = {
            'action_func': Var(var_type='action_func', source={'params': {'choices': action_func_choices}, 'sample': 'categorical'}),
            'lr': Var(var_type='lr', source={'params': {'low': lr_tuple[0], 'high': lr_tuple[1], 'log': True}, 'sample': 'float'}),
            'wd': Var(var_type='wd', source={'params': {'low': wd_tuple[0], 'high': wd_tuple[1], 'log': True}, 'sample': 'float'}),
            'batch_size': Var(var_type='batch_size', source={'params': {'choices': batch_size_choices}, 'sample': 'categorical'}),
            'n_epoch': Var(var_type='n_epoch', source={'params': {'choices': n_epoch_choices}, 'sample': 'categorical'})
        }
        return TrainSampler(INIT_TRAIN_DICT=INIT_TRAIN_DICT)

    def mock_wrapper_init(self, trial, default_dict, sampler, device):
        action = default_dict['action_func']
        model = action(device=device)

        self.assertIsInstance(model, nn.Module)
        self.assertEqual(model(torch.randn(size=[10, 16])).shape, torch.Size([10, 10]))

        #Mock trainer that will update all the parameters to zero
        trainer = Mock()
        trainer.side_effect = lambda: [param.data.fill_(0) for param in model.parameters()]

        #Ensure the model's state dict gets updated with zeros
        for param in model.parameters():
            param.data.fill_(0)

        execution_set = Mock()
        execution_set.yield_execution_set.return_value = (trainer, model)

        #Return the execution set
        return execution_set

    def test_update(self):
        param_sets = self.construct_mock_trials()
        for i,param_set in enumerate(param_sets):
            sampler = Sampler(INIT_DICT=self.get_init_dict())
            top_k_trials = []
            model_train_window = param_set['model_train_window']
            distributions = param_set['distributions']
            xyz = param_set['xyz']
            for params in param_set['params']:
                trial = Mock(spec=FrozenTrial)
                trial.params = params
                top_k_trials.append(trial)
            #Mock the wrapper
            Wrapper = Mock()
            Wrapper.side_effect = self.mock_wrapper_init
            sampler.update(top_k_trials=top_k_trials,
                           model_train_window=model_train_window,
                           device=None)
            for var_type in sampler.train_keys():
                var = sampler.get_var(var_type)
                self.assertEqual(
                    var.distribution(), distributions[var_type],
                    msg = f"{i}th param_set: {var.var_type}.distribution(): {var.distribution()} != \nExpected distribution for {var_type}: {distributions[var_type]}"
                )
            for action_func in sampler.action_funcs():
                action = sampler.get_action(action_func)
                assert isinstance(action, Action), f"{i}th param_set: action: {action} != An instance of 'Action'"
                for var_type in action.keys():
                    var_retrieved_from_sampler = sampler.get_var(action_func, var_type)
                    var = action.get_var(var_type)
                    self.assertEqual(
                        var_retrieved_from_sampler, var,
                        msg = f"{i}th param_set: sampler.get_var(action_func, var_type): {var_retrieved_from_sampler} != \n action[var_type]: {var}"
                    )
                    #Since 'var_type' is from 'action.keys(), it may NOT be in distributions.
                    #IF SO, then continue.
                    if var_type not in distributions.keys():
                        continue
                    try:
                        distribution = distributions[var_type].get(action_func, None)
                    except Exception as e:
                        print(f"distributions: {distributions}, distributions[var_type]: {distributions[var_type]} Exception: {e}")
                    if distribution is None:
                        continue
                    self.assertEqual(
                        var.distribution(), distribution,
                        msg = f"{i}th param_set::{action_func}:: {var.var_type}.distribution(): {var.distribution()} != \nExpected distribution: {distribution}"
                    )
            self.assertEqual(self.produce_saved_bools(sampler), param_set['xyz'])
            self.assertEqual(sampler.is_fixed(), param_set.get('fixed', False))

    def produce_saved_bools(self, sampler: Sampler):
        action_sampler = sampler.action_sampler
        default_action = action_sampler.get_action(action_default)
        dropout_action = action_sampler.get_action(action_dropout)
        dropout_variant_action = action_sampler.get_action(action_dropout_variant)
        default_saved = default_action.action_state is not None
        dropout_saved = dropout_action.action_state is not None
        dropout_variant_saved = dropout_variant_action.action_state is not None

        if default_saved:
            for param_tensor in default_action.action_state.values():
                if param_tensor.numel() > 0:  # Check tensor is not empty
                    default_saved &= torch.all(param_tensor == 0).item()
        if dropout_saved:
            for param_tensor in dropout_action.action_state.values():
                if param_tensor.numel() > 0:  # Check tensor is not empty
                    dropout_saved &= torch.all(param_tensor == 0).item()

        if dropout_variant_saved:
            for param_tensor in dropout_variant_action.action_state.values():
                if param_tensor.numel() > 0:  # Check tensor is not empty
                    dropout_variant_saved &= torch.all(param_tensor == 0).item()

        return (default_saved, dropout_saved, dropout_variant_saved)

    def construct_mock_trials(self):
        #Constructs a list of mock trials based on the provided param_sets structure
        param_sets = [
            {
                'params': [
                    {'action_func': 1, 'activation': 1, 'dropout_p': 0.78, 'lr': 1e-1, 'n_epoch': 9, 'wd': 1e-2, 'batch_size': 6},
                    {'action_func': 0, 'rand_number': 1, 'lr': 1e-2, 'n_epoch': 11, 'wd': 5e-2, 'batch_size': 7},
                    {'action_func': 1, 'activation': 2, 'dropout_p': 0.123, 'lr': 1e-3, 'n_epoch': 13, 'wd': 1e-3, 'batch_size': 5},
                    {'action_func': 0, 'rand_number': 1, 'lr': 1e-9, 'n_epoch': 19, 'wd': 2e-2, 'batch_size': 8},
                    {'action_func': 0, 'rand_number': 3, 'lr': 2e-3, 'n_epoch': 0, 'wd': 4e-2, 'batch_size': 6},
                    #Addendum.
                    {'action_func': 2, 'activation': 0, 'dropout_p': 0.0, 'lr': 1e-3, 'n_epoch': 13, 'wd': 1e-3, 'batch_size': 5}
                ],
                'distributions': {
                    'action_func': [action_default, action_default, action_default, action_dropout, action_dropout, action_dropout_variant],
                    'rand_number': {action_default: [2, 2, 4]},
                    'activation': {action_dropout: [nn.PReLU, nn.SiLU], action_dropout_variant: [nn.ReLU]},
                    'batch_size': [2 ** exp for exp in [5, 5, 6, 6, 7, 8]],
                    'n_epoch': [1, 10, 12, 14, 14, 20],
                    'dropout_p': {action_dropout: {'low': 0.123, 'high': 0.78}, action_dropout_variant: {'low': 0.0, 'high': 0.0}},
                    'lr': {'low': 1e-9, 'high': 1e-1, 'log': True},
                    'wd': {'low': 1e-3, 'high': 5e-2, 'log': True}
                },
                'model_train_window': 1,
                'xyz': (False, True, False)
            },
            {
                'params': [
                    {'action_func': 0, 'rand_number': 3, 'lr': 1e-2, 'n_epoch': 11, 'wd': 1e-2, 'batch_size': 0},

                    #Addendum
                    {'action_func': 2, 'activation': 1, 'dropout_p': 0.13, 'lr': 1e-3, 'n_epoch': 13, 'wd': 3e-2, 'batch_size': 8},

                    {'action_func': 1, 'activation': 0, 'dropout_p': 0.012, 'lr': 5e-1, 'n_epoch': 9, 'wd': 1e-1, 'batch_size': 1},
                    {'action_func': 1, 'activation': 2, 'dropout_p': 0.13, 'lr': 1e-3, 'n_epoch': 13, 'wd': 3e-2, 'batch_size': 8},
                    {'action_func': 0, 'rand_number': 7, 'lr': 1e-3, 'n_epoch': 19, 'wd': 4e-4, 'batch_size': 3},
                    {'action_func': 0, 'rand_number': 9, 'lr': 2e-3, 'n_epoch': 4, 'wd': 5e-3, 'batch_size': 4},
                ],
                'distributions': {
                    'action_func': [action_default, action_default, action_default, action_dropout, action_dropout, action_dropout_variant],
                    'rand_number': {action_default: [4, 8, 10]},
                    'activation': {action_dropout: [nn.ReLU, nn.SiLU], action_dropout_variant: [nn.PReLU]},
                    'batch_size': [2 ** exp for exp in [0, 1, 3, 4, 8, 8]],
                    'n_epoch': [5, 10, 12, 14, 14, 20],
                    'dropout_p': {action_dropout: {'low': 0.012, 'high': 0.13}, action_dropout_variant: {'low': 0.13, 'high': 0.13}},
                    'lr': {'low': 1e-3, 'high': 5e-1, 'log': True},
                    'wd': {'low': 4e-4, 'high': 1e-1, 'log': True}
                },
                'model_train_window': 1,
                'xyz': (True, False, False)
            },
            {
                'params': [
                    {'action_func': 1, 'activation': 0, 'dropout_p': 0.532, 'lr': 3e-1, 'n_epoch': 17, 'wd': 6e-5, 'batch_size': 8},
                    {'action_func': 0, 'rand_number': 5, 'lr': 3e-1, 'n_epoch': 17, 'wd': 6e-5, 'batch_size': 8},

                    #Addendum
                    {'action_func': 2, 'activation': 2, 'dropout_p': 0.532, 'lr': 3e-1, 'n_epoch': 17, 'wd': 6e-5, 'batch_size': 8},

                    {'action_func': 1, 'activation': 0, 'dropout_p': 0.532, 'lr': 3e-1, 'n_epoch': 17, 'wd': 6e-5, 'batch_size': 8},
                    {'action_func': 0, 'rand_number': 5, 'lr': 3e-1, 'n_epoch': 17, 'wd': 6e-5, 'batch_size': 8},
                    {'action_func': 0, 'rand_number': 5, 'lr': 3e-1, 'n_epoch': 17, 'wd': 6e-5, 'batch_size': 8},
                ],
                'distributions': {
                    'action_func': [action_default, action_default, action_default, action_dropout, action_dropout, action_dropout_variant],
                    'rand_number': {action_default: [6]},
                    'activation': {action_dropout: [nn.ReLU], action_dropout_variant: [nn.SiLU]},
                    'batch_size': [2 ** 8],
                    'n_epoch': [18],
                    'dropout_p': {action_dropout: {'low': 0.532, 'high': 0.532}, action_dropout_variant: {'low': 0.532, 'high': 0.532}},
                    'lr': {'low': 3e-1, 'high': 3e-1, 'log': True},
                    'wd': {'low': 6e-5, 'high': 6e-5, 'log': True}
                },
                'model_train_window': 3,
                'xyz': (True, True, True)
            },
            {
                'params': [
                    {'action_func': 1, 'activation': 1, 'dropout_p': 0.176, 'lr': 1e-1, 'n_epoch': 10, 'wd': 5e-2, 'batch_size': 1},

                    #Addendum
                    {'action_func': 2, 'activation': 1, 'dropout_p': 0.111, 'lr': 1e-1, 'n_epoch': 10, 'wd': 5e-2, 'batch_size': 1},

                    {'action_func': 1, 'activation': 0, 'dropout_p': 0.176, 'lr': 1e-3, 'n_epoch': 14, 'wd': 2e-1, 'batch_size': 2},
                    {'action_func': 0, 'rand_number': 1, 'lr': 1e-9, 'n_epoch': 19, 'wd': 7e-3, 'batch_size': 3}
                ],
                'distributions': {
                    'action_func': [action_default, action_dropout, action_dropout, action_dropout_variant],
                    'rand_number': {action_default: [2]},
                    'activation':  {action_dropout: [nn.ReLU, nn.PReLU], action_dropout_variant: [nn.PReLU]},
                    'batch_size': [2, 2, 4, 8],
                    'n_epoch': [11, 11, 15, 20],
                    'dropout_p': {action_dropout: {'low': 0.176, 'high': 0.176}, action_dropout_variant: {'low': 0.111, 'high': 0.111}},
                    'lr': {'low': 1e-9, 'high': 1e-1, 'log': True},
                    'wd': {'low': 7e-3, 'high': 2e-1, 'log': True}
                },
                'model_train_window': 2,
                'xyz': (False, True, True)
            },
            {
                'params': [
                    {'action_func': 1, 'activation': 2, 'dropout_p': 0.85, 'lr': 1e-3, 'n_epoch': 10, 'wd': 1e-4, 'batch_size': 7},
                    {'action_func': 1, 'activation': 2, 'dropout_p': 0.85, 'lr': 1e-3, 'n_epoch': 10, 'wd': 1e-4, 'batch_size': 7},
                    {'action_func': 1, 'activation': 2, 'dropout_p': 0.85, 'lr': 1e-3, 'n_epoch': 10, 'wd': 1e-4, 'batch_size': 7}
                ],
                'distributions': {
                    'action_func': [action_dropout],
                    'activation': {action_dropout: [nn.SiLU]},
                    'batch_size': [128],
                    'n_epoch': [11],
                    'dropout_p': {action_dropout: {'low': 0.85, 'high': 0.85}},
                    'lr': {'low': 1e-3, 'high': 1e-3, 'log': True},
                    'wd': {'low': 1e-4, 'high': 1e-4, 'log': True}
                },
                'model_train_window': 100,
                'xyz': (False, True, False),
                'fixed': True
            }
        ]
        return param_sets

    def get_init_dict(self):
        action_func_choices = [action_default, action_dropout, action_dropout_variant]
        rand_number_choices = list(range(1,11))
        activation_choices = [nn.ReLU, nn.PReLU, nn.SiLU]
        dropout_p_min = 0.0
        dropout_p_max = 1.0
        lr_min = 1e-9
        lr_max = 5.0
        wd_min = 1e-9
        wd_max = 1.0
        n_epoch_choices = list(range(1,21))
        batch_size_choices = [2**exp for exp in range(9)]

        # Construct INIT_DICT for each param_set
        init_dict = {
            'action_func': {
                'params': {'choices': action_func_choices},
                'sample': 'categorical'
            },
            'rand_number': {
                'params': {'choices': rand_number_choices},
                'sample': 'categorical'
            },
            'activation': {
                'params': {'choices': activation_choices},
                'sample': 'categorical'
            },
            'dropout_p': {
                'params': {'low': dropout_p_min, 'high': dropout_p_max},
                'sample': 'float'
             },
            'lr': {
                'params': {'low': lr_min, 'high': lr_max, 'log': True},
                'sample': 'float'
            },
            'n_epoch': {
                'params': {'choices': n_epoch_choices},
                'sample': 'categorical'
            },
            'wd': {
                'params': {'low': wd_min, 'high': wd_max, 'log': True},
                'sample': 'float'
            },
            'batch_size': {
                'params': {'choices': batch_size_choices},
                'sample': 'categorical'
            }
        }
        return init_dict

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

[I 2024-09-27 21:37:28,848] A new study created in memory with name: no-name-3c70d645-bbd1-410f-8dca-135f2768efd5
.[I 2024-09-27 21:37:28,878] A new study created in memory with name: no-name-cdeb2f45-e5d1-49bb-a425-69e12310a82b
.[I 2024-09-27 21:37:28,899] A new study created in memory with name: no-name-5f552c3c-5f35-48ca-b52d-a6e3e8b38782
.[I 2024-09-27 21:37:28,918] A new study created in memory with name: no-name-0c7b7a56-5ceb-40d8-ac24-7039893e4378
.[I 2024-09-27 21:37:28,942] A new study created in memory with name: no-name-459b5a67-ac65-402f-86f6-93497b8357ab
.[I 2024-09-27 21:37:28,949] A new study created in memory with name: no-name-d03f4ef1-f258-4106-93f2-0ae1bf89edea
.[I 2024-09-27 21:37:28,960] A new study created in memory with name: no-name-1e94e4d8-c021-46fc-8449-419e0a7fcdbe
.[I 2024-09-27 21:37:29,011] A new study created in memory with name: no-name-e5a8d25d-c2c3-4d77-8063-0cc4de16d6e3
.[I 2024-09-27 21:37:29,020] A new study created in memory with name: no-name-6d6

#Class: Wrapper

In [9]:
class Wrapper:
    def __init__(self, trial, sampler: Sampler, device: torch.device, default_dict: dict = {}):
        #Perform general-error checking
        self._error_check(trial=trial, sampler=sampler, default_dict=default_dict)

        #Initialize all the member variables.
        self.trial = trial
        self.default_dict = default_dict
        self.default_keys = set(default_dict.keys())
        self.sampler = sampler
        self.train_keys = set(self.sampler.train_keys()) if self.sampler else self.default_keys
        self.action_funcs = set(self.sampler.action_funcs()) if self.sampler else set([self.default_dict['action_func'].action_func])
        self.sampled_vars = set()
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #print("*************************************************************************************")
        #print(f"default_dict: {default_dict}")
        #print("*************************************************************************************")

    def _error_check(self, trial: Trial, sampler: Sampler, default_dict: dict):
        #trial type check:
        if trial is not None and not isinstance(trial, (Trial, FrozenTrial)):
            raise ValueError(f"Input 'trial' must either be 'None' or an instance in (optuna::Trial, optuna::FrozenTrial) but found: {trial}")

        #sampler type check:
        if sampler is not None and not isinstance(sampler, Sampler):
            raise ValueError(f"Input 'sampler' must either be None of type Sampler but found: {sampler}")

        #Make sure there's something to be sampled:
        if not default_dict and not sampler:
            raise ValueError(f"Both default_dict and sampler are empty. Nothing to be sampled.")

        #Make sure that IF 'action_func' exists in 'default_dict',
        #               THEN it maps to a properly built & loadable Action object
        manual_action = default_dict.get('action_func', None)
        manual_action_valid = isinstance(manual_action, Action) and manual_action.is_loadable()
        if not sampler and not manual_action:
            raise ValueError("Input 'sampler' is None but 'action_func' does NOT exist as a key in 'default_dict'. Note that 'action_func' is one parameter required for every Auto-hpo process")
        #IF ('action_func'->obj) exists, THEN check on its value.
        if manual_action and not manual_action_valid:
            msg = "which is NOT an Action object" if not isinstance(manual_action, Action) else "which is NOT loadable"
            raise ValueError(f"Input 'default_dict' has 'action_func' as a key but default_dict['action_func']->{manual_action}, {msg}")

    def sample(self, var_type: str, default_val = None):
        #ERROR CHECKS
        #No double sampling.
        var_type_in_default = var_type in self.default_keys
        if var_type in self.sampled_vars:
            raise ValueError(f"var_type: {var_type} has already been sampled. Only one sampling per var_type in each trial allowed. Note the following var_type(s) have already been sampled: {self.sampled_vars}")

        sampled_val = None
        #IF var_type is in default, THEN sampled_val <- default_dict[var_type]
        if var_type in self.default_keys:
            sampled_val = self.default_dict[var_type]
        elif self.sampler is None or var_type not in self.train_keys:
            #IF var_type is NOT in default_dict AND (self.sampler is None OR var_type is NOT in self.sampler),
            #THEN sampled_val <- default_val
            sampled_val = default_val
        else: #All False, THEN sampled_val <- sampled value from self.sampler
            sampled_val = self.sampler.sample(var_type=var_type, trial=self.trial, default_val=default_val)

        #Add 'var_type' to the set of sampled variables.
        self.sampled_vars.add(var_type)

        return sampled_val

    def yield_execution_set(self):
        #Dictionary of (key->default_val) for key among common parameters that appear in 'dls_func'
        default_dls_map = {
            'batch_size': 64,
            'img_size': 224,
            'normalize': True,
            'summary': True
        }
        #dls_func <- sampled 'dls_func' from Sampler.
        dls_func = self.sample('dls_func')
        try:
            #dls_sig <- signature parameters of 'dls_func'
            dls_sig = set(signature(dls_func).parameters.keys())
        except Exception as e:
            raise ValueError(f"dls_func: {dls_func}, retrieving the dls_sig from it via 'set(signature(dls_func).parameters.keys())' resulted in the following: {e}")

        #dls <- DataLoaders created from 'dls_func' with sampled signatures
        dls = dls_func(
            **{key:self.sample(key, default_dls_map[key]) for key in dls_sig}
        )

        #action <- Action object encapsulating the logic of sampled recipe
        action = self.sample('action_func')
        #action = self.sample(self.default_dict)
        #action_func, action_state <- Useful properties of 'action'.
        action_func = action.action_func
        action_state = action.action_state
        #model <- Model yielded from action. Note it's loaded onto the input 'device'.
        model = action(device=self.device)
        if not isinstance(model, nn.Module):
            raise ValueError(f"Model generated by Action: {action_func.__name__} must be of type 'nn.Module' but found: {model}")

        loss_func = self.sample('loss_func')
        learner = Learner(dls=dls,
                          model=model,
                          loss_func=loss_func() if isinstance(loss_func, nn.Module) else loss_func,
                          #opt_func=self.sample('opt_func'), NOT IMPLEMENTED YET!!!
                          metrics=self.sample('metric')
                          )

        #Sample on whether to freeze the learner or not ONLY WHEN there's no saved 'action_state' for the given action_func
        if action_state is None and self.sample('freeze', False):
            learner.freeze()
        else: #IF (action_state is not None) OR (action_state is None but sampled 'freeze' states False)
              #THEN .unfreeze().
              learner.unfreeze()

        #Barebone cbs list:
        cbs = []
        #Fill the list:
        if self.sample('gradient_clip',False): #Apply Gradient clipping, if exists and is True.
            cbs.append(GradientClip(max_norm=self.sample('max_norm', 0.1))) #If exists, then 'max_norm' MUST exist, hence no safety guarantee.
        if self.trial is None:
            cbs.append(EarlyStoppingCallback(monitor='valid_loss',
                                             min_delta=self.sample('min_delta', 0.1),
                                             patience=self.sample('patience', 15)))
        else: #IF trial is a type of optuna.Trail, then apply PruningCallback.
            cbs.append(FastAIPruningCallback(trial=self.trial, monitor='valid_loss'))

        #Make the action and return it.
        trainer = None
        if self.sample('one_cycle', True):
            lr_low = self.sample('lr_low', 1e-3)
            lr_high = self.sample('lr_high', lr_low)
            trainer = partial(learner.fit_one_cycle,
                              n_epoch=self.sample('n_epoch', 10),
                              lr_max=slice(lr_low, lr_high),
                              wd=self.sample('wd', 0.01), pct_start=self.sample('pct_start',0.3),
                              cbs=cbs)
        else:
            trainer = partial(learner.fit,
                               n_epoch=self.sample('n_epoch', 10),
                               lr=self.sample('lr', 1e-3),
                               wd=self.sample('wd', 0.01),
                               cbs=cbs)

        #Return the execution_set := (trainer, learner)
        #NOTE: 'trainer' Executes training on the learer, when called.
        return (trainer, learner)

#Test: Wrapper

In [10]:
from torch.utils.data import Dataset

class PolynomialDataset(Dataset):
    def __init__(self, coeffs: torch.tensor, input_min: float, input_max: float, input_size: int):
        #coeffs.shape == [degree+1, 1]
        self.coeffs = coeffs.clone().detach()
        #inputs.shape == [inputs]
        self.inputs = torch.empty(size=(input_size,)).uniform_(input_min, input_max)
        #Preprocess the output and retrieve it upon request.
        self.outputs = self._f(X=self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]

    def __len__(self):
        if self.inputs.size(0) != self.outputs.size(0):
            return ValueError(f"len(inputs): {self.inputs.size(0)} != len(outputs): {self.outputs.size(0)}")
        return self.inputs.size(0)

    def _f(self, X):
        #power.shape == [1, degree+1]
        powers = torch.arange(self.coeffs.size(0)).unsqueeze(0)
        Y = self.inputs.unsqueeze(-1)**powers
        return torch.matmul(Y, self.coeffs).squeeze(-1)

if __name__ == '__main__':
    degree = 6
    coeffs_min, coeffs_max = -10.0, 10.0
    coeffs = torch.empty(size=(degree+1,)).uniform_(coeffs_min, coeffs_max)

    for c in coeffs:
        print(f"c: {c}")

    train_dset = PolynomialDataset(coeffs=coeffs, input_min=-100.0, input_max=100.0, input_size=8000)
    valid_dset = PolynomialDataset(coeffs=coeffs, input_min=-300.12, input_max=500.78, input_size=2000)

    #Test:
    print(f"Number of elements in Dataset; train_dset: {len(train_dset)}, valid_dset: {len(valid_dset)}")
    print()

    print(f"train first input: {train_dset[0][0]}, train first output: {train_dset[0][1]}")
    print()

    print(f"valid first input: {valid_dset[0][0]}, valid first output: {valid_dset[0][1]}")
    print()

c: -4.371528625488281
c: -9.356799125671387
c: 3.873828649520874
c: 0.6507885456085205
c: 5.898780822753906
c: 3.7403082847595215
c: 7.500274181365967
Number of elements in Dataset; train_dset: 8000, valid_dset: 2000

train first input: 99.92070007324219, train first output: 7502502559744.0

valid first input: 256.47650146484375, valid first output: 2139003100856320.0



In [11]:
if __name__ == '__main__':
    """
    Action functions:
    """
    class ModelDefault(nn.Module):
        def __init__(self, activation: nn.Module, preprocess: bool, start_neurons: int):
            super().__init__()
            self.preprocess = preprocess
            if self.preprocess: start_neurons = max(start_neurons,degree+1)
            self.net = nn.Sequential(
                nn.Linear(in_features=degree+1 if self.preprocess else 1,
                        out_features=start_neurons),
                nn.BatchNorm1d(num_features=start_neurons),
                activation(),
                nn.Linear(in_features=start_neurons,out_features=start_neurons*2),
                nn.BatchNorm1d(num_features=start_neurons*2),
                activation(),
                nn.Linear(in_features=start_neurons*2,out_features=1)
            )

        def _preprocess(self, X):
            N = X.size(0)
            #X <- [64,1]x[1,7] == [64,7], where each index (i,j) yields X[i]**j.
            X = X.unsqueeze(-1)**torch.arange(degree+1).unsqueeze(0)
            if X.size() != torch.Size([N,degree+1]):
                raise ValueError(f"X.size() MUST be torch.Size([{N},{degree+1}]) but found X.size()=={X.size()}")
            return X

        def forward(self, X):
            if X.dim() != 1:
                X = X.reshape(-1)

            if self.preprocess:
                X = self._preprocess(X)
            else:
                X = X.unsqueeze(-1)
            return self.net(X)

    class ModelDropout(nn.Module):
        def __init__(self, activation: nn.Module, dropout_p: int, preprocess: int, start_neurons: Int):
            super().__init__()
            self.preprocess = preprocess
            if self.preprocess: start_neurons = max(start_neurons,degree+1)
            self.net = nn.Sequential(
                nn.Linear(in_features=degree+1 if self.preprocess else 1,
                        out_features=start_neurons),
                nn.BatchNorm1d(num_features=start_neurons),
                activation(),
                nn.Dropout(p=dropout_p),
                nn.Linear(in_features=start_neurons,out_features=start_neurons*2),
                nn.BatchNorm1d(num_features=start_neurons*2),
                activation(),
                nn.Linear(in_features=start_neurons*2,out_features=1)
            )

        def _preprocess(self, X):
            N = X.size(0)
            #X <- [64,1]x[1,7] == [64,7], where each index (i,j) yields X[i]**j.
            X = X.unsqueeze(-1)**torch.arange(degree+1).unsqueeze(0)
            if X.size() != torch.Size([N,degree+1]):
                raise ValueError(f"X.size() MUST be torch.Size([{N},{degree+1}]) but found X.size()=={X.size()}")
            return X

        def forward(self, X):
            if X.dim() != 1:
                X = X.reshape(-1)

            if self.preprocess:
                X = self._preprocess(X)
            else:
                X = X.unsqueeze(-1)
            return self.net(X)

    def action_default(activation: nn.Module, preprocess: bool, start_neurons: int, device: torch.device):
        if start_neurons < 1:
            raise ValueError(f"start_neurons MUST be a positive integer but found start_neurons: {start_neurons}")
        return ModelDefault(activation=activation,
                            preprocess=preprocess,
                            start_neurons=start_neurons).to(device)

    def action_dropout(activation: nn.Module, dropout_p: int, preprocess: bool, start_neurons: int, device: torch.device):
        if start_neurons < 1:
            raise ValueError(f"start_neurons MUST be a positive integer but found start_neurons: {start_neurons}")
        if dropout_p < 0.0 or dropout_p >= 1.0:
            raise ValueError(f"0.0 <= dropout_p < 1.0 required but found dropout_p: {dropout_p}")
        return ModelDropout(activation=activation,
                            dropout_p=dropout_p,
                            preprocess=preprocess,
                            start_neurons=start_neurons).to(device)

    #Define DataLoaders and training process
    def get_dls(batch_size: int):
        train_dataloader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
        valid_dataloader = DataLoader(valid_dset, batch_size=batch_size, shuffle=True)

        dls = DataLoaders(train_dataloader, valid_dataloader)
        return dls

    def custom_mse(Y_hat, Y):
        Y_hat = Y_hat.reshape(-1)
        Y = Y.reshape(-1)
        if Y_hat.size() != Y.size():
            raise ValueError(f"Y_hat.size(): {Y_hat.size()} != Y.size(): {Y.size()}")
        return nn.MSELoss()(Y_hat, Y)

In [13]:
class TestWrapper(unittest.TestCase):
    def setUp(self):
        self.INIT_DICT = {
            'dls_func': {
                'params': {'choices': [get_dls]},
                'sample': 'categorical'
            },
            'freeze': {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            'lr_low': {
                'params': {'low': 1e-7, 'high': 9e-4, 'log': True},
                'sample': 'float'
            },
            'lr_high': {
                'params': {'low': 9e-4, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            'lr': {
                'params': {'low': 1e-6, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            'wd': {
                'params': {'choices': [1e-4, 1e-3, 1e-2, 1e-1]},
                'sample': 'categorical'
            },
            'gradient_clip': {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            'max_norm': {
                'params': {'low': 0.0, 'high': 15.0, 'log': False},
                'sample': 'float'
            },
            'one_cycle': {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            'pct_start': {
                'params': {'low': 0.10, 'high': 0.95,  'log': False, 'step': 0.05},
                'sample': 'float'
            },
            'n_epoch': {
                'params': {'choices': [5, 10, 15]},
                'sample': 'categorical'
            },
            'action_func': {
                'params': {'choices': [action_default, action_dropout]},
                'sample': 'categorical'
            },
            'activation': {
                'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]},
                'sample': 'categorical'
            },
            'loss_func': {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            'metric': {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            'preprocess': {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            'start_neurons': {
                'params': {'choices': [1, 2, 4, 8, 16, 32]},
                'sample': 'categorical'
            },
            'dropout_p': {
                'params': {'low': 0.01, 'high': 0.95,  'log': False, 'step': 0.05},
                'sample': 'float'
            },
            'batch_size': {
                'params': {'choices': [1, 2, 4, 8, 16, 32, 64]},
                'sample': 'categorical'
            }
        }

        self.trial = optuna.create_study(direction='minimize').ask()
        self.device = torch.device('cpu')

    def test_initialization(self):
        sampler = Sampler(INIT_DICT=self.INIT_DICT)
        self.assertTrue(set(sampler.sig_keys())==set(['activation', 'dropout_p', 'preprocess', 'start_neurons']),
                        msg=f"sampler.sig_keys(): {set(sampler.sig_keys())}")
        self.assertTrue(set(sampler.train_keys())==set(self.INIT_DICT.keys())-set(['activation', 'dropout_p', 'preprocess', 'start_neurons']),
                        msg=f"sampler.train_keys(): {set(sampler.train_keys())}")
        self.assertTrue(set(sampler.action_funcs())==set([action_default, action_dropout]),
                        msg=f"sampler.action_funcs(): {set(sampler.action_funcs())}")

        sampled_dict = dict()
        for key in sampler.train_keys():
            sampled_dict[key] = sampler.sample(key, self.trial, default_val=None)

        wrapper = Wrapper(trial=self.trial, sampler=sampler, device=self.device, default_dict={})
        self.assertTrue(sampled_dict, wrapper.default_dict)

    def test_yield_execution_set_trial(self):
        sampler = Sampler(INIT_DICT=self.INIT_DICT)
        wrapper = Wrapper(trial=self.trial, sampler=sampler, device=self.device, default_dict={})
        trainer, learner = wrapper.yield_execution_set()
        trainer()
        self.assertIsInstance(learner, Learner)
        model = learner.model
        self.assertIsInstance(model, nn.Module)
        self.assertEqual(model(torch.randn(size=[64,1])).shape, torch.Size([64,1]))

    def test_yield_execution_set_frozen_trial(self):
        #Create a FrozenTrial with all parameters pre-determined
        trial_params = {
            'dls_func': get_dls,
            'freeze': True,
            'lr_low': 1e-6,
            'lr_high': 1e-2,
            'lr': 1e-3,
            'wd': 1e-4,
            'gradient_clip': False,
            'max_norm': 10.0,
            'one_cycle': True,
            'pct_start': 0.5,
            'n_epoch': 10,
            'action_func': action_dropout,
            'activation': nn.ReLU,
            'loss_func': custom_mse,
            'metric': custom_mse,
            'preprocess': False,
            'start_neurons': 16,
            'dropout_p': 0.5,
            'batch_size': 64
        }
        frozen_trial = optuna.trial.FrozenTrial(
            number=0,
            trial_id=0,
            state=optuna.trial.TrialState.COMPLETE,
            value=None,
            datetime_start=None,
            datetime_complete=None,
            params=trial_params,
            distributions={
                'dls_func': optuna.distributions.CategoricalDistribution(choices=[0]),
                'freeze': optuna.distributions.CategoricalDistribution(choices=[0, 1]),
                'lr_low': optuna.distributions.FloatDistribution(low=1e-7, high=9e-4, log=True),
                'lr_high': optuna.distributions.FloatDistribution(low=9e-4, high=1e-1, log=True),
                'lr': optuna.distributions.FloatDistribution(low=1e-6, high=1e-1, log=True),
                'wd': optuna.distributions.CategoricalDistribution(choices=[0, 1, 2, 3]),
                'gradient_clip': optuna.distributions.CategoricalDistribution(choices=[0, 1]),
                'max_norm': optuna.distributions.FloatDistribution(low=0.0, high=15.0),
                'one_cycle': optuna.distributions.CategoricalDistribution(choices=[0, 1]),
                'pct_start': optuna.distributions.FloatDistribution(low=0.10, high=0.95, step=0.05),
                'n_epoch': optuna.distributions.CategoricalDistribution(choices=[0,1,2]),
                'action_func': optuna.distributions.CategoricalDistribution(choices=[0,1]),
                'activation': optuna.distributions.CategoricalDistribution(choices=[0,1,2]),
                'loss_func': optuna.distributions.CategoricalDistribution(choices=[0]),
                'metric': optuna.distributions.CategoricalDistribution(choices=[0]),
                'preprocess': optuna.distributions.CategoricalDistribution(choices=[0, 1]),
                'start_neurons': optuna.distributions.CategoricalDistribution(choices=[0,1,2,3,4,5]),
                'dropout_p': optuna.distributions.FloatDistribution(low=0.01, high=0.95, step=0.05),
                'batch_size': optuna.distributions.CategoricalDistribution(choices=[0])
            },
            user_attrs={},
            system_attrs={},
            intermediate_values={}
        )
        sampler = Sampler(INIT_DICT=self.INIT_DICT)

        copy_params = dict()
        #Change all the actual values in frozen_trial.params to indices(if applicable)
        for var_type in set(sampler.train_keys())|sampler.sig_keys():
            val = frozen_trial.params[var_type]
            if var_type in sampler.sig_keys():
                copy_params[var_type] = sampler.convert_val_to_idx(1, var_type, val)
                var = sampler.get_var(1, var_type)
            else:
                copy_params[var_type] = sampler.convert_val_to_idx(var_type, val)
                var = sampler.get_var(var_type)
                self.assertEqual(
                    copy_params[var_type],
                    var.convert_val_to_idx(val),
                    msg=f"For Var:{var_type}, {var_type}.distribution: {var.distribution()}, copy_params[{var_type}]: {copy_params[var_type]} != \n{var_type}.convert_val_to_idx({val}): {var.convert_val_to_idx(val)}"
                )
                self.assertEqual(
                    val,
                    var.convert_idx_to_val(copy_params[var_type]),
                    msg=f"For Var:{var_type}, {var_type}.distribution: {var.distribution(indices=True)}, frozen_trial.params[{var_type}]: {val} != \n{var_type}.convert_idx_to_val({copy_params[var_type]}): {var.convert_idx_to_val(copy_params[var_type])}"
                )

        #Overwrite all the params of frozen_trial
        frozen_trial.params |= copy_params

        #CHECK: frozen_trial.params ONLY contain values that are either int or float
        for var_type,idx in frozen_trial.params.items():
            if not isinstance(idx, (int,float)):
                raise ValueError(f"var_type:{var_type}->idx:{idx} BUT type(idx):{type(idx)}!=(int,float)")

        #Initialize the Wrapper with the pre-determined FrozenTrial
        wrapper = Wrapper(trial=frozen_trial, sampler=sampler, device=self.device, default_dict={})

        #Generate the trainer and learner, and execute the trainer
        trainer, learner = wrapper.yield_execution_set()
        trainer()

        #Verify that the frozen trial parameters are used without modification
        self.assertEqual(frozen_trial.params, copy_params, msg=f"Frozen_trial.params: {frozen_trial.params} != \n original_frozen_trial_params: {copy_params}")

        #Update sampler with output of frozen trial
        sampler.update(top_k_trials=[frozen_trial], model_train_window=1, device=self.device)

        #ONLY one action_func shall be available.
        action_func_var = sampler.get_var('action_func')
        self.assertEqual(action_func_var.distribution(), [action_dropout],
                        msg=f"action_func_var.distribution(): {action_func_var.distribution()} != [action_dropout]"
        )

        action = sampler.get_action(action_dropout)
        action_compare_to = sampler.get_action(1)
        action_compare_to_two = sampler.action_sampler.get_action(action_dropout)
        self.assertEqual(action, action_compare_to)
        self.assertEqual(action, action_compare_to_two)

        #Go through each distribution of each var object, compare it against the value(s) from copy_params
        for var_type in sampler.train_keys():
            var = sampler.get_var(var_type)
            distribution = var.distribution()
            compare_to = [var.convert_idx_to_val(copy_params[var_type])] if var.sample_method == 'categorical' else distribution | {'low': copy_params[var_type], 'high': copy_params[var_type]}
            self.assertEqual(
                distribution,
                compare_to,
                msg=f"{var_type}.distribution: {distribution} differs from expected distribution: {compare_to}"
            )

        for var_type in action.keys():
            var = action.get_var(var_type)
            compare_to_var = sampler.get_var(action.action_func, var_type)
            self.assertEqual(var, compare_to_var)
            distribution = var.distribution()
            compare_to = [var.convert_idx_to_val(copy_params[var_type])] if var.sample_method == 'categorical' else distribution | {'low': copy_params[var_type], 'high': copy_params[var_type]}
            self.assertEqual(
                distribution,
                compare_to,
                msg=f"action_func: {action.action_func}::{var_type}.distribution: {distribution} differs from expected distribution: {compare_to}"
            )

    def test_initialization_errors(self):
        study = optuna.create_study(direction='minimize')

        #Error Test Case 1: Invalid trial type
        sampler = Sampler(INIT_DICT=self.INIT_DICT)
        with self.assertRaises(ValueError) as context:
            Wrapper(trial="invalid_trial_type", sampler=sampler, device=self.device)
        self.assertIn("Input 'trial' must either be 'None' or an instance in (optuna::Trial, optuna::FrozenTrial)", str(context.exception),
                    msg=str(context.exception))

        #Error Test Case 2: Invalid sampler type
        sampler = Sampler(INIT_DICT=self.INIT_DICT)
        with self.assertRaises(ValueError) as context:
            Wrapper(trial=study.ask(), sampler="invalid_sampler_type", device=self.device)
        self.assertIn("Input 'sampler' must either be None of type Sampler", str(context.exception),
                    msg=str(context.exception))

        #Error Test Case 3: Both default_dict and sampler are empty
        sampler = Sampler(INIT_DICT=self.INIT_DICT)
        with self.assertRaises(ValueError) as context:
            Wrapper(trial=None, sampler=None, device=self.device, default_dict={})
        self.assertIn("Both default_dict and sampler are empty. Nothing to be sampled.", str(context.exception),
                    msg=str(context.exception))

        #Error Test Case 4: 'action_func' in default_dict is not a valid Action object
        sampler = Sampler(INIT_DICT=self.INIT_DICT)
        with self.assertRaises(ValueError) as context:
            Wrapper(trial=study.ask(), sampler=sampler, device=self.device, default_dict={'action_func': 'invalid_action'})
        self.assertIn("Input 'default_dict' has 'action_func' as a key", str(context.exception),
                    msg=str(context.exception))
        self.assertIn("which is NOT an Action object", str(context.exception),
                    msg=str(context.exception))

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

[I 2024-09-27 21:41:36,969] A new study created in memory with name: no-name-a909559c-4ba6-49b2-8160-1fdf188ba6c2
.[I 2024-09-27 21:41:36,986] A new study created in memory with name: no-name-7e5a35ca-4863-4bec-b9d5-b06cc6d1c099
[I 2024-09-27 21:41:36,993] A new study created in memory with name: no-name-42216ffd-0343-49e7-82ba-b99fb80a1527
.[I 2024-09-27 21:41:37,004] A new study created in memory with name: no-name-06c7bd67-4123-496c-bb62-01e95910ae9e
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:689: UserWarning: The distribution is specified by [0.01, 0.95] and step=0.05, but the range is not divisible by `step`. It will be replaced by [0.01, 0.91].
  warnings.warn(


epoch,train_loss,valid_loss,custom_mse,time
0,4378513696958552289050624.000000,740999726901489132038941738795008.000000,740999726901489132038941738795008.000000,00:01
1,4194657881079650705211392.000000,740999726901489132038941738795008.000000,740999726901489132038941738795008.000000,00:01
2,4257143920786332597616640.000000,740999804272741587375208919990272.000000,740999804272741587375208919990272.000000,00:01
3,4232131576974263204184064.000000,740999804272741587375208919990272.000000,740999804272741587375208919990272.000000,00:01
4,4471765447015667893010432.000000,740999726901489132038941738795008.000000,740999726901489132038941738795008.000000,00:01
5,4216089827159163535360000.000000,740999726901489132038941738795008.000000,740999726901489132038941738795008.000000,00:01
6,4228602772479037797302272.000000,740999804272741587375208919990272.000000,740999804272741587375208919990272.000000,00:02
7,4225133343441299643039744.000000,740999726901489132038941738795008.000000,740999726901489132038941738795008.000000,00:02
8,4251674172938101563850752.000000,740999804272741587375208919990272.000000,740999804272741587375208919990272.000000,00:01
9,4279408564422547722993664.000000,740999726901489132038941738795008.000000,740999726901489132038941738795008.000000,00:01


epoch,train_loss,valid_loss,custom_mse,time
0,4123985810459884047564800.000000,740999726901489132038941738795008.000000,740999726901489132038941738795008.000000,00:02
1,4291826970179044223483904.000000,740999804272741587375208919990272.000000,740999804272741587375208919990272.000000,00:01
2,4273658656648697225412608.000000,740999804272741587375208919990272.000000,740999804272741587375208919990272.000000,00:00
3,4311486875905976329830400.000000,740999804272741587375208919990272.000000,740999804272741587375208919990272.000000,00:00
4,4277494138264148053590016.000000,740999726901489132038941738795008.000000,740999726901489132038941738795008.000000,00:00
5,4283636615810317182566400.000000,740999726901489132038941738795008.000000,740999726901489132038941738795008.000000,00:00
6,4306541130881589108015104.000000,740999804272741587375208919990272.000000,740999804272741587375208919990272.000000,00:01
7,4303046914031501906542592.000000,740999804272741587375208919990272.000000,740999804272741587375208919990272.000000,00:02
8,4311133793695190482944000.000000,740999726901489132038941738795008.000000,740999726901489132038941738795008.000000,00:01
9,4206313052800097473003520.000000,740999804272741587375208919990272.000000,740999804272741587375208919990272.000000,00:01


.[I 2024-09-27 21:42:09,549] A new study created in memory with name: no-name-7b09ba62-44e7-4a6b-98f9-828fb7fd0a13


epoch,train_loss,valid_loss,custom_mse,time
0,4362664773265098115383296.000000,740999881643994042711476101185536.000000,740999881643994042711476101185536.000000,00:02
1,4179488316382786116124672.000000,740999649530236676702674557599744.000000,740999649530236676702674557599744.000000,00:02
2,4309172385985478084526080.000000,740999649530236676702674557599744.000000,740999649530236676702674557599744.000000,00:02
3,4432190839909285567135744.000000,740999959015246498047743282380800.000000,740999959015246498047743282380800.000000,00:04
4,4144926035517682057478144.000000,740999726901489132038941738795008.000000,740999726901489132038941738795008.000000,00:02


.
----------------------------------------------------------------------
Ran 4 tests in 46.112s

OK


#Class: HpoTrainer

In [14]:
class HpoTrainer:
    def __init__(self, sampler: Sampler, n_trials: int, device: torch.device, prev_top_k_trials: list = [],
                 pruner: optuna.pruners = None, timeout: int = 14400, eta: int = 2, learner_bar: bool = False, learner_logging: bool = False):
        """
        Class Explanation: HpoTrainer is a TEMPORARY wrapper for running a single HPO process.
        """
        self.sampler = sampler
        self.device = device
        self.eta = eta
        self.timeout = timeout
        self.learner_bar = learner_bar
        self.learner_logging = learner_logging

        #Enqueue ALL the previous_top_k_trials.
        #NOTE: This allows the optimizer to efficiently search the new space.
        self.prior_trials = prev_top_k_trials

        # Use the user-defined n_trials for new trials (ignore the prior trial count)
        self.n_trials = n_trials

        # Select a pruner if not specified
        choices = (optuna.pruners.HyperbandPruner, optuna.pruners.SuccessiveHalvingPruner)
        self.pruner = random.choice(choices) if pruner is None else pruner

    def run(self):
        print("*****************RUNNING: HPO PROCESS*****************")

        # To get logging info on the HPO process:
        optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

        # Create the study object with pruner
        study = optuna.create_study(
            direction='minimize',
            pruner=self.pruner(reduction_factor=self.eta)
        )

        # Enqueue the prior trials (these will run first)
        for trial in self.prior_trials:
            study.enqueue_trial(trial.params)

        # Run the optimization for the user-defined n_trials (new trials only)
        study.optimize(func=self.objective, n_trials=self.n_trials, timeout=self.timeout)

        # Return the optimized study
        return study

    def objective(self, trial):
        # Sanity check
        if not isinstance(trial, (Trial, FrozenTrial)):
            raise ValueError(f"Input 'trial' MUST be an instance of optuna::Trial OR optuna::FrozenTrial but found trial: {trial}. "
                             f"NOTE: FrozenTrial is NOT allowed for optimization via HpoTrainer!")

        #Initialize a wrapper with the trial and sampler
        sample_wrapper = Wrapper(trial=trial, sampler=self.sampler, device=self.device)

        #Unpack the execution set yielded by the 'text_vars'
        trainer, learner = sample_wrapper.yield_execution_set()

        #Run the trainer with logging and progress bar management
        if not self.learner_bar and not self.learner_logging:
            with learner.no_bar(), learner.no_logging():
                trainer()
        elif not self.learner_bar:
            with learner.no_bar():
                trainer()
        elif not self.learner_logging:
            with learner.no_logging():
                trainer()
        else:
            trainer()

        #Return the validation loss (index 0 of learner.validate())
        return learner.validate()[0]

#Test: HpoTrainer

In [15]:
class TestHpoTrainer(unittest.TestCase):
    def setUp(self):
        self.INIT_DICT = {
            'dls_func': {
                'params': {'choices': [get_dls]},
                'sample': 'categorical'
            },
            'freeze': {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            'lr_low': {
                'params': {'low': 1e-7, 'high': 9e-4, 'log': True},
                'sample': 'float'
            },
            'lr_high': {
                'params': {'low': 9e-4, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            'lr': {
                'params': {'low': 1e-6, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            'wd': {
                'params': {'choices': [1e-4, 1e-3, 1e-2, 1e-1]},
                'sample': 'categorical'
            },
            'gradient_clip': {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            'max_norm': {
                'params': {'low': 0.0, 'high': 15.0, 'log': False},
                'sample': 'float'
            },
            'one_cycle': {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            'pct_start': {
                'params': {'low': 0.10, 'high': 0.95,  'log': False, 'step': 0.05},
                'sample': 'float'
            },
            'n_epoch': {
                'params': {'choices': list(range(1,11))},
                'sample': 'categorical'
            },
            'action_func': {
                'params': {'choices': [action_default, action_dropout]},
                'sample': 'categorical'
            },
            'activation': {
                'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]},
                'sample': 'categorical'
            },
            'loss_func': {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            'metric': {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            'preprocess': {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            'start_neurons': {
                'params': {'choices': [1, 2, 4, 8, 16, 32]},
                'sample': 'categorical'
            },
            'dropout_p': {
                'params': {'low': 0.01, 'high': 0.95,  'log': False, 'step': 0.05},
                'sample': 'float'
            },
            'batch_size': {
                'params': {'choices': [64]},
                'sample': 'categorical'
            }
        }
        self.device = torch.device('cpu')

    def test_initialization(self):
        sampler = Sampler(INIT_DICT=self.INIT_DICT)
        hpo_trainer = HpoTrainer(
            sampler = sampler,
            n_trials = 2,
            device = self.device
        )
        self.assertIsInstance(hpo_trainer, HpoTrainer)
        self.assertIsInstance(hpo_trainer.sampler, Sampler)
        self.assertEqual(hpo_trainer.n_trials, 2)
        self.assertEqual(hpo_trainer.device, self.device)
        self.assertEqual(hpo_trainer.eta, 2)
        self.assertEqual(hpo_trainer.timeout, 14400)
        self.assertEqual(hpo_trainer.eta, 2)
        self.assertEqual(hpo_trainer.learner_bar, False)
        self.assertEqual(hpo_trainer.learner_logging, False)

    def test_run(self):
        n_trials = [4, 2, 1]
        top_ks = random.choices(list(range(1,10)), k=len(n_trials))
        for n,k in zip(n_trials,top_ks):
            sampler = Sampler(INIT_DICT=self.INIT_DICT)
            hpo_trainer = HpoTrainer(
                sampler = sampler,
                n_trials = n,
                device = self.device
            )
            study = hpo_trainer.run()
            prev_trials = study.trials
            top_k = min(len(prev_trials), k)
            top_k_trials = sorted(
                prev_trials,
                key=lambda trial: trial.value if trial.value is not None else float('inf')
            )[:top_k]
            sampler.update(
                top_k_trials=top_k_trials,
                model_train_window=2,
                device=self.device
        )
        print("****************************************")
        print(sampler)
        print("****************************************")
        print()

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.[I 2024-09-27 21:42:23,203] A new study created in memory with name: no-name-08479a87-affc-411d-8f77-d30466e06c61


*****************RUNNING: HPO PROCESS*****************
A new study created in memory with name: no-name-08479a87-affc-411d-8f77-d30466e06c61


/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:689: UserWarning: The distribution is specified by [0.01, 0.95] and step=0.05, but the range is not divisible by `step`. It will be replaced by [0.01, 0.91].
  warnings.warn(


[I 2024-09-27 21:42:25,821] Trial 0 finished with value: 7.409996495302367e+32 and parameters: {'dls_func': 0, 'batch_size': 0, 'action_func': 1, 'dropout_p': 0.46, 'start_neurons': 0, 'preprocess': 1, 'activation': 2, 'loss_func': 0, 'metric': 0, 'freeze': 1, 'gradient_clip': 1, 'one_cycle': 1, 'n_epoch': 3, 'lr': 0.01128815872068802, 'wd': 0}. Best is trial 0 with value: 7.409996495302367e+32.


Trial 0 finished with value: 7.409996495302367e+32 and parameters: {'dls_func': 0, 'batch_size': 0, 'action_func': 1, 'dropout_p': 0.46, 'start_neurons': 0, 'preprocess': 1, 'activation': 2, 'loss_func': 0, 'metric': 0, 'freeze': 1, 'gradient_clip': 1, 'one_cycle': 1, 'n_epoch': 3, 'lr': 0.01128815872068802, 'wd': 0}. Best is trial 0 with value: 7.409996495302367e+32.


...

epoch,train_loss,valid_loss,custom_mse,time
0,4153461689877038849064960.000000,740999649530236676702674557599744.000000,740999649530236676702674557599744.000000,00:00
1,4237744287089065486974976.000000,740999649530236676702674557599744.000000,740999649530236676702674557599744.000000,00:00
2,4242977974259228268822528.000000,740999726901489132038941738795008.000000,740999726901489132038941738795008.000000,00:00
3,4334917699644101280923648.000000,740999726901489132038941738795008.000000,740999726901489132038941738795008.000000,00:00
4,4377585306916967625523200.000000,740999804272741587375208919990272.000000,740999804272741587375208919990272.000000,00:00
5,4385744244174694129860608.000000,740999804272741587375208919990272.000000,740999804272741587375208919990272.000000,00:01
6,4278789157344197694455808.000000,740999804272741587375208919990272.000000,740999804272741587375208919990272.000000,00:00
7,4200388477418299038105600.000000,740999649530236676702674557599744.000000,740999649530236676702674557599744.000000,00:00
8,4283284398290659790815232.000000,740999649530236676702674557599744.000000,740999649530236676702674557599744.000000,00:00
9,4226241589237602974695424.000000,740999804272741587375208919990272.000000,740999804272741587375208919990272.000000,00:00


.[I 2024-09-27 21:43:15,626] A new study created in memory with name: no-name-f7f9f9d2-3cd1-41cd-b5e4-9ee76648b8f9


A new study created in memory with name: no-name-f7f9f9d2-3cd1-41cd-b5e4-9ee76648b8f9
A new study created in memory with name: no-name-f7f9f9d2-3cd1-41cd-b5e4-9ee76648b8f9
A new study created in memory with name: no-name-f7f9f9d2-3cd1-41cd-b5e4-9ee76648b8f9


epoch,train_loss,valid_loss,custom_mse,time
0,3292479798132599995498496.000000,740999417416479310693873014013952.000000,740999417416479310693873014013952.000000,00:10
1,4765693849473522826477568.000000,740999959015246498047743282380800.000000,740999959015246498047743282380800.000000,00:09
2,3987002306581149979246592.000000,740999881643994042711476101185536.000000,740999881643994042711476101185536.000000,00:10
3,3819679961539062884466688.000000,740999726901489132038941738795008.000000,740999726901489132038941738795008.000000,00:09
4,4615235287279320082415616.000000,740999107931469489348804289232896.000000,740999107931469489348804289232896.000000,00:09


.
----------------------------------------------------------------------
Ran 6 tests in 102.172s

OK


#ConfigSpace

In [28]:
class ConfigSpace:
    def __init__(self, INIT_DICT: dict=None):
        """
        Initialize the hyperparameter configuration.

        Parameters:
        - INIT_DICT (dict, optional): An initial hyperparameter configuration dictionary.
        """
        if INIT_DICT is not None:
            self.config = INIT_DICT.copy()
        else:
            self.config = {}

    def __len__(self):
        return len(self.config)

    def get_params(self, names):
        if isinstance(names, str):
            names = [names]
        return {name: self.config[name] for name in names}

    def set_float(self, name: str, low: float, high: float, *, step=None, log=False):
        """
        Define a float hyperparameter.

        Parameters:
        - name (str): The name of the hyperparameter.
        - low (float): The lower bound of the hyperparameter.
        - high (float): The upper bound of the hyperparameter.
        - step (float, optional): The step size between values.
        - log (bool, optional): Whether to sample the value logarithmically.
        """
        params = {'low': low, 'high': high, 'log': log}
        if step is not None:
            params['step'] = step

        self.config[name] = {
            'params': params,
            'sample': 'float'
        }

    def set_int(self, name: str, low: int, high: int, *, step=1, log=False):
        """
        Define an integer hyperparameter.

        Parameters:
        - name (str): The name of the hyperparameter.
        - low (int): The lower bound of the hyperparameter.
        - high (int): The upper bound of the hyperparameter.
        - step (int, optional): The step size between values.
        - log (bool, optional): Whether to sample the value logarithmically.
        """
        params = {'low': low, 'high': high, 'step': step, 'log': log}

        self.config[name] = {
            'params': params,
            'sample': 'int'
        }

    def set_categorical(self, name: str, choices: list):
        """
        Define a categorical hyperparameter.

        Parameters:
        - name (str): The name of the hyperparameter.
        - choices (list): A list of possible values.
        """
        self.config[name] = {
            'params': {'choices': choices},
            'sample': 'categorical'
        }

#Class: Auto

In [29]:
class Auto:
    def __init__(self, INIT_ARG, storage_dir: str = '/content/trial_storage/'):
        if not isinstance(INIT_ARG, (dict, ConfigSpace)):
            raise ValueError(f"Input 'INIT_ARG' must be an instance of (dict, ConfigSpace) but found: {type(INIT_ARG)}")
        INIT_DICT = INIT_ARG.config if isinstance(INIT_ARG, ConfigSpace) else INIT_ARG
        self.sampler = Sampler(INIT_DICT=INIT_DICT)
        if self.sampler.is_fixed():
            raise ValueError("Input 'INIT_ARG' is completely determined from the beginning. Nothing to optimize for")

        self.prev_top_k_trials = []
        self.storage_dir = storage_dir
        os.makedirs(self.storage_dir, exist_ok=True)
        self._n_study = None

    def optimize(self, n_auto: int, top_k: int, model_train_window: int, device: torch.device):
        list_n_trials = self._get_list_n_trials(n_auto=n_auto)
        print("*****************************************************************")
        print(f"n_auto: {n_auto}, list_n_trials: {list_n_trials}")
        print()

        for idx, n_trials in enumerate(list_n_trials):
            hpo_pointer = HpoTrainer(
                sampler=self.sampler,
                n_trials=n_trials,
                device=device,
                prev_top_k_trials=self.prev_top_k_trials
            )
            optimized_study = hpo_pointer.run()
            self._update(
                idx=idx,
                optimized_study=optimized_study,
                top_k=top_k,
                model_train_window=model_train_window,
                device=device
            )
            self._n_study = len(list_n_trials)

    def num_study(self):
        return self._n_study

    def get_study(self, idx: int):
        if self._n_study is None:
            raise SyntaxError(".visualization_trials is requested w/o saved data. Call .optimize() first and once it's finished, request visualization again.")
        if idx < 0 or idx >= self._n_study:
            raise ValueError(f"Input 'idx':{idx} must satisfy 0 <= idx < {self._n_study} but failed.")

        return self._load_study_from_pickle(idx)

    def _get_list_n_trials(self, n_auto: int):
        """
        ACCEPTS:
        n_auto := Positive integer power of 2. IF n_auto <= 1, THEN n_auto <- 2
                                               IF n_auto > 1 but NOT a power of 2, THEN n_auto <- (nearest power of 2) - 1.
        RETURNS:
        Exponential distribution of [2^m, 2^(m-1), ..., 2^1, 2^0] + (n_auto==power of 2)*[2^0] trials returned.
        """
        n_auto = max(n_auto, 2)
        pow2 = n_auto & (n_auto-1) == 0

        output = [1<<e for e in range(math.ceil(math.log2(n_auto)) - 1, -1, -1)]

        if pow2:
            output.append(1)

        return output

    def _update(self, idx: int, optimized_study: optuna.study, top_k: int, model_train_window: int, device: torch.device):
        prev_trials = optimized_study.trials
        top_k = min(top_k, len(prev_trials))
        model_train_window = min(model_train_window, top_k)

        top_k_trials = sorted(prev_trials, key=lambda trial: trial.value if trial.value is not None else float('inf'))[:top_k]

        #Save the whole study, not just top_k_trials
        self._save_study_to_pickle(idx, optimized_study)
        self.prev_top_k_trials = top_k_trials

        self.sampler.update(
            top_k_trials=top_k_trials,
            model_train_window=model_train_window,
            device=device
        )

    def _save_study_to_pickle(self, idx: int, study: optuna.study):
        save_path = os.path.join(self.storage_dir, f'study_{idx}.pkl')
        with gzip.open(save_path, 'wb') as f:
            pickle.dump(study, f)
        print(f"Saved study to: {save_path}")

    def _load_study_from_pickle(self, idx: int):
        load_path = os.path.join(self.storage_dir, f'study_{idx}.pkl')
        if not os.path.exists(load_path):
            raise FileNotFoundError(f"No study found at: {load_path}")

        with gzip.open(load_path, 'rb') as f:
            study = pickle.load(f)
        print(f"Loaded study from: {load_path}")
        return study

#Prep: FINAL_TEST

In [30]:
if __name__ == '__main__':
    from torch.utils.data import Dataset

    class PolynomialDataset(Dataset):
        def __init__(self, coeffs: torch.tensor, input_min: float, input_max: float, input_size: int):
            #coeffs.shape == [degree+1, 1]
            self.coeffs = coeffs.clone().detach()
            #inputs.shape == [inputs]
            self.inputs = torch.empty(size=(input_size,)).uniform_(input_min, input_max)
            #Preprocess the output and retrieve it upon request.
            self.outputs = self._f(X=self.inputs)

        def __getitem__(self, idx):
            return self.inputs[idx], self.outputs[idx]

        def __len__(self):
            if self.inputs.size(0) != self.outputs.size(0):
                return ValueError(f"len(inputs): {self.inputs.size(0)} != len(outputs): {self.outputs.size(0)}")
            return self.inputs.size(0)

        def _f(self, X):
            #power.shape == [1, degree+1]
            powers = torch.arange(self.coeffs.size(0)).unsqueeze(0)
            Y = self.inputs.unsqueeze(-1)**powers
            return torch.matmul(Y, self.coeffs).squeeze(-1)

    degree = 6
    coeffs_min, coeffs_max = -10.0, 10.0
    coeffs = torch.empty(size=(degree+1,)).uniform_(coeffs_min, coeffs_max)

    for c in coeffs:
        print(f"c: {c}")

    train_dset = PolynomialDataset(coeffs=coeffs, input_min=-100.0, input_max=100.0, input_size=8000)
    valid_dset = PolynomialDataset(coeffs=coeffs, input_min=-300.12, input_max=500.78, input_size=2000)

    #Test:
    print(f"Number of elements in Dataset; train_dset: {len(train_dset)}, valid_dset: {len(valid_dset)}")
    print()

    print(f"train first input: {train_dset[0][0]}, train first output: {train_dset[0][1]}")
    print()

    print(f"valid first input: {valid_dset[0][0]}, valid first output: {valid_dset[0][1]}")
    print()

    """
    Action functions:
    """
    class ModelDefault(nn.Module):
        def __init__(self, activation: nn.Module, preprocess: bool, start_neurons: int):
            super().__init__()
            self.preprocess = preprocess
            if self.preprocess: start_neurons = max(start_neurons,degree+1)
            self.net = nn.Sequential(
                nn.Linear(in_features=degree+1 if self.preprocess else 1,
                        out_features=start_neurons),
                nn.BatchNorm1d(num_features=start_neurons),
                activation(),
                nn.Linear(in_features=start_neurons,out_features=start_neurons*2),
                nn.BatchNorm1d(num_features=start_neurons*2),
                activation(),
                nn.Linear(in_features=start_neurons*2,out_features=1)
            )

        def _preprocess(self, X):
            N = X.size(0)
            #X <- [64,1]x[1,7] == [64,7], where each index (i,j) yields X[i]**j.
            X = X.unsqueeze(-1)**torch.arange(degree+1).unsqueeze(0)
            if X.size() != torch.Size([N,degree+1]):
                raise ValueError(f"X.size() MUST be torch.Size([{N},{degree+1}]) but found X.size()=={X.size()}")
            return X

        def forward(self, X):
            if X.dim() != 1:
                raise ValueError(f"X.dim() MUST be 1 but found X.dim()=={X.dim()}")

            if self.preprocess:
                X = self._preprocess(X)
            else:
                X = X.unsqueeze(-1)
            return self.net(X)

    class ModelDropout(nn.Module):
        def __init__(self, activation: nn.Module, dropout_p: int, preprocess: int, start_neurons: Int):
            super().__init__()
            self.preprocess = preprocess
            if self.preprocess: start_neurons = max(start_neurons,degree+1)
            self.net = nn.Sequential(
                nn.Linear(in_features=degree+1 if self.preprocess else 1,
                        out_features=start_neurons),
                nn.BatchNorm1d(num_features=start_neurons),
                activation(),
                nn.Dropout(p=dropout_p),
                nn.Linear(in_features=start_neurons,out_features=start_neurons*2),
                nn.BatchNorm1d(num_features=start_neurons*2),
                activation(),
                nn.Linear(in_features=start_neurons*2,out_features=1)
            )

        def _preprocess(self, X):
            N = X.size(0)
            #X <- [64,1]x[1,7] == [64,7], where each index (i,j) yields X[i]**j.
            X = X.unsqueeze(-1)**torch.arange(degree+1).unsqueeze(0)
            if X.size() != torch.Size([N,degree+1]):
                raise ValueError(f"X.size() MUST be torch.Size([{N},{degree+1}]) but found X.size()=={X.size()}")
            return X

        def forward(self, X):
            if X.dim() != 1:
                raise ValueError(f"X.dim() MUST be 1 but found X.dim()=={X.dim()}")

            if self.preprocess:
                X = self._preprocess(X)
            else:
                X = X.unsqueeze(-1)
            return self.net(X)

    def action_default(activation: nn.Module, preprocess: bool, start_neurons: int, device: torch.device):
        if start_neurons < 1:
            raise ValueError(f"start_neurons MUST be a positive integer but found start_neurons: {start_neurons}")
        return ModelDefault(activation=activation,
                            preprocess=preprocess,
                            start_neurons=start_neurons).to(device)

    def action_dropout(activation: nn.Module, dropout_p: int, preprocess: bool, start_neurons: int, device: torch.device):
        if start_neurons < 1:
            raise ValueError(f"start_neurons MUST be a positive integer but found start_neurons: {start_neurons}")
        if dropout_p < 0.0 or dropout_p >= 1.0:
            raise ValueError(f"0.0 <= dropout_p < 1.0 required but found dropout_p: {dropout_p}")
        return ModelDropout(activation=activation,
                            dropout_p=dropout_p,
                            preprocess=preprocess,
                            start_neurons=start_neurons).to(device)

    #Define DataLoaders and training process
    def get_dls(batch_size: int):
        train_dataloader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
        valid_dataloader = DataLoader(valid_dset, batch_size=batch_size, shuffle=True)

        dls = DataLoaders(train_dataloader, valid_dataloader)
        return dls

    # Now you can use this DataLoaders object with Learner
    # Example usage:
    train_dls, valid_dls = get_dls(batch_size=64)

    print(f"len(train_dls): {len(train_dls)}")
    print(f"len(valid_dls): {len(valid_dls)}")
    print()

    print("*****CHECKING data integrity of returned 'dls'******")
    for blob in iter(train_dls):
        if not isinstance(blob, tuple):
            raise ValueError(f"Sampled 'blob' from 'iter(train_dls)' MUST have a type 'tuple' but found: {type(blob)}")
        inputs, outputs = blob
        if inputs.size() != outputs.size():
            raise ValueError(f"train_dls contains (inputs,outputs) where inputs.size(): {inputs.size()} != outputs.size(): {outputs.size()}")

    for blob in iter(valid_dls):
        if not isinstance(blob, tuple):
            raise ValueError(f"Sampled 'blob' from 'iter(valid_dls)' MUST have a type 'tuple' but found: {type(blob)}")
        inputs, outputs = blob
        if inputs.size() != outputs.size():
            raise ValueError(f"valid_dls contains (inputs,outputs) where inputs.size(): {inputs.size()} != outputs.size(): {outputs.size()}")

    inputs, _ = next(iter(train_dls))
    print(inputs.size())
    inputs, _ = next(iter(valid_dls))
    print(inputs.size())

    model_default = ModelDefault(activation=nn.ReLU, preprocess=False, start_neurons=7)
    model_dropout = ModelDropout(activation=nn.ReLU, dropout_p=0.5, preprocess=False, start_neurons=7)

    print("*********************model_default*********************")
    print(model_default)
    print()
    print("*********************model_dropout*********************")
    print(model_dropout)
    print()

    learner = Learner(
        dls=get_dls(batch_size=64),
        model=ModelDefault(activation=nn.ReLU, preprocess=False, start_neurons=7),
        loss_func=custom_mse,
        metrics=[custom_mse]
    )
    learner.fit_one_cycle(n_epoch=10, lr_max=slice(1e-6,1e-3), pct_start=0.3, wd=0.01)

c: -9.434732437133789
c: -6.319784164428711
c: -3.426584005355835
c: 5.433167457580566
c: 5.635682106018066
c: 7.37066650390625
c: 3.9590811729431152
Number of elements in Dataset; train_dset: 8000, valid_dset: 2000

train first input: -60.87129211425781, train first output: 195320954880.0

valid first input: -150.75328063964844, valid first output: 45901380845568.0

len(train_dls): 125
len(valid_dls): 32

*****CHECKING data integrity of returned 'dls'******
torch.Size([64])
torch.Size([64])
*********************model_default*********************
ModelDefault(
  (net): Sequential(
    (0): Linear(in_features=1, out_features=7, bias=True)
    (1): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=7, out_features=14, bias=True)
    (4): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=14, out_features=1, bias=True)
  )
)

********************

epoch,train_loss,valid_loss,custom_mse,time
0,1194567551369037616250880.000000,202310737776475548106725282283520.000000,202310737776475548106725282283520.000000,00:00
1,1235158819069701067374592.000000,202310699090849320438591691685888.000000,202310699090849320438591691685888.000000,00:00
2,1198489069751769730383872.000000,202310737776475548106725282283520.000000,202310737776475548106725282283520.000000,00:00
3,1202638001901285545082880.000000,202310699090849320438591691685888.000000,202310699090849320438591691685888.000000,00:00
4,1199996946964607410372608.000000,202310737776475548106725282283520.000000,202310737776475548106725282283520.000000,00:00
5,1185710664255459741925376.000000,202310718433662434272658486984704.000000,202310718433662434272658486984704.000000,00:00
6,1214767672878472068333568.000000,202310718433662434272658486984704.000000,202310718433662434272658486984704.000000,00:00
7,1196759255149295232352256.000000,202310699090849320438591691685888.000000,202310699090849320438591691685888.000000,00:00
8,1188722959916621281361920.000000,202310737776475548106725282283520.000000,202310737776475548106725282283520.000000,00:00
9,1227454853460730040025088.000000,202310718433662434272658486984704.000000,202310718433662434272658486984704.000000,00:00


In [31]:
if __name__ == '__main__':
    def custom_mse(Y_hat, Y):
        Y_hat = Y_hat.reshape(-1)
        Y = Y.reshape(-1)
        if Y_hat.size() != Y.size():
            raise ValueError(f"Y_hat.size(): {Y_hat.size()} != Y.size(): {Y.size()}")
        return nn.MSELoss()(Y_hat, Y)

    INIT_DICT = {
        'dls_func': {
            'params': {'choices': [get_dls]},
            'sample': 'categorical'
        },
        'freeze': {
            'params': {'choices': [True, False]},
            'sample': 'categorical'
        },
        'lr_low': {
            'params': {'low': 1e-7, 'high': 9e-4, 'log': True},
            'sample': 'float'
        },
        'lr_high': {
            'params': {'low': 9e-4, 'high': 1e-1, 'log': True},
            'sample': 'float'
        },
        'lr': {
            'params': {'low': 1e-6, 'high': 1e-1, 'log': True},
            'sample': 'float'
        },
        'wd': {
            'params': {'choices': [1e-4, 1e-3, 1e-2, 1e-1]},
            'sample': 'categorical'
        },
        'gradient_clip': {
            'params': {'choices': [True, False]},
            'sample': 'categorical'
        },
        'max_norm': {
            'params': {'low': 0.0, 'high': 15.0, 'log': False},
            'sample': 'float'
        },
        'one_cycle': {
            'params': {'choices': [True, False]},
            'sample': 'categorical'
        },
        'pct_start': {
            'params': {'low': 0.10, 'high': 0.95,  'log': False, 'step': 0.05},
            'sample': 'float'
        },
        'n_epoch': {
            'params': {'choices': [5, 10, 15]},
            'sample': 'categorical'
        },
        'action_func': {
            'params': {'choices': [action_default, action_dropout]},
            'sample': 'categorical'
        },
        'activation': {
            'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]},
            'sample': 'categorical'
        },
        'loss_func': {
            'params': {'choices': [custom_mse]},
            'sample': 'categorical'
        },
        'metric': {
            'params': {'choices': [custom_mse]},
            'sample': 'categorical'
        },
        'preprocess': {
            'params': {'choices': [True, False]},
            'sample': 'categorical'
        },
        'start_neurons': {
            'params': {'choices': [1, 2, 4, 8, 16, 32,]},
            'sample': 'categorical'
        },
        'dropout_p': {
            'params': {'low': 0.05, 'high': 0.95,  'log': False, 'step': 0.05},
            'sample': 'float'
        },
        'batch_size': {
            'params': {'choices': [32, 64, 128]},
            'sample': 'categorical'
        }
    }

    config = ConfigSpace(INIT_DICT=INIT_DICT)

#FINAL_TEST

In [32]:
if __name__ == '__main__':
    auto = Auto(INIT_ARG=config)
    auto.optimize(n_auto=10, top_k=2, model_train_window=2, device=torch.device('cpu'))

    #Print out the optimized sampler.
    print()
    print("*****************************************************************")
    print()
    print(auto.sampler)
    print()
    print("*****************************************************************")
    print()

    #TEST: Optuna's visualization capabilities
    print(auto.num_study())
    study = auto.get_study(2)
    #One can now perform all the optuna visualizations as one sees fit
    optuna.visualization.plot_optimization_history(study)

[I 2024-09-27 22:26:04,851] A new study created in memory with name: no-name-d5bd08df-2137-4aba-94b3-f483e2ba54e8


*****************************************************************
n_auto: 10, list_n_trials: [8, 4, 2, 1]

*****************RUNNING: HPO PROCESS*****************
A new study created in memory with name: no-name-d5bd08df-2137-4aba-94b3-f483e2ba54e8
A new study created in memory with name: no-name-d5bd08df-2137-4aba-94b3-f483e2ba54e8
A new study created in memory with name: no-name-d5bd08df-2137-4aba-94b3-f483e2ba54e8
A new study created in memory with name: no-name-d5bd08df-2137-4aba-94b3-f483e2ba54e8
A new study created in memory with name: no-name-d5bd08df-2137-4aba-94b3-f483e2ba54e8
A new study created in memory with name: no-name-d5bd08df-2137-4aba-94b3-f483e2ba54e8
A new study created in memory with name: no-name-d5bd08df-2137-4aba-94b3-f483e2ba54e8
A new study created in memory with name: no-name-d5bd08df-2137-4aba-94b3-f483e2ba54e8


[I 2024-09-27 22:26:09,317] Trial 0 finished with value: 2.0231069909084932e+32 and parameters: {'dls_func': 0, 'batch_size': 1, 'action_func': 0, 'start_neurons': 1, 'preprocess': 0, 'activation': 2, 'loss_func': 0, 'metric': 0, 'freeze': 1, 'gradient_clip': 0, 'max_norm': 12.091289544526283, 'one_cycle': 1, 'n_epoch': 0, 'lr': 4.195944382743136e-06, 'wd': 3}. Best is trial 0 with value: 2.0231069909084932e+32.


Trial 0 finished with value: 2.0231069909084932e+32 and parameters: {'dls_func': 0, 'batch_size': 1, 'action_func': 0, 'start_neurons': 1, 'preprocess': 0, 'activation': 2, 'loss_func': 0, 'metric': 0, 'freeze': 1, 'gradient_clip': 0, 'max_norm': 12.091289544526283, 'one_cycle': 1, 'n_epoch': 0, 'lr': 4.195944382743136e-06, 'wd': 3}. Best is trial 0 with value: 2.0231069909084932e+32.
Trial 0 finished with value: 2.0231069909084932e+32 and parameters: {'dls_func': 0, 'batch_size': 1, 'action_func': 0, 'start_neurons': 1, 'preprocess': 0, 'activation': 2, 'loss_func': 0, 'metric': 0, 'freeze': 1, 'gradient_clip': 0, 'max_norm': 12.091289544526283, 'one_cycle': 1, 'n_epoch': 0, 'lr': 4.195944382743136e-06, 'wd': 3}. Best is trial 0 with value: 2.0231069909084932e+32.
Trial 0 finished with value: 2.0231069909084932e+32 and parameters: {'dls_func': 0, 'batch_size': 1, 'action_func': 0, 'start_neurons': 1, 'preprocess': 0, 'activation': 2, 'loss_func': 0, 'metric': 0, 'freeze': 1, 'gradient

[I 2024-09-27 22:26:18,141] Trial 1 finished with value: 2.0231069909084932e+32 and parameters: {'dls_func': 0, 'batch_size': 1, 'action_func': 0, 'start_neurons': 2, 'preprocess': 1, 'activation': 0, 'loss_func': 0, 'metric': 0, 'freeze': 1, 'gradient_clip': 0, 'max_norm': 3.8670344240117567, 'one_cycle': 1, 'n_epoch': 2, 'lr': 0.0051711553180824035, 'wd': 2}. Best is trial 0 with value: 2.0231069909084932e+32.


...

*****************RUNNING: HPO PROCESS*****************
A new study created in memory with name: no-name-8fcc33fc-a634-4d93-bba6-6d9ccf551209
A new study created in memory with name: no-name-8fcc33fc-a634-4d93-bba6-6d9ccf551209
A new study created in memory with name: no-name-8fcc33fc-a634-4d93-bba6-6d9ccf551209
A new study created in memory with name: no-name-8fcc33fc-a634-4d93-bba6-6d9ccf551209
A new study created in memory with name: no-name-8fcc33fc-a634-4d93-bba6-6d9ccf551209
A new study created in memory with name: no-name-8fcc33fc-a634-4d93-bba6-6d9ccf551209
A new study created in memory with name: no-name-8fcc33fc-a634-4d93-bba6-6d9ccf551209
A new study created in memory with name: no-name-8fcc33fc-a634-4d93-bba6-6d9ccf551209
A new study created in memory with name: no-name-8fcc33fc-a634-4d93-bba6-6d9ccf551209
A new study created in memory with name: no-name-8fcc33fc-a634-4d93-bba6-6d9ccf551209
A new study created in memory with name: no-name-8fcc33fc-a634-4d93-bba6-6d9ccf551209

[I 2024-09-27 22:27:32,904] Trial 0 finished with value: 2.0231073777647555e+32 and parameters: {'dls_func': 0, 'batch_size': 1, 'action_func': 0, 'loss_func': 0, 'metric': 0, 'gradient_clip': 0, 'max_norm': 12.626330626734708, 'one_cycle': 1, 'n_epoch': 0, 'lr': 7.357972600141196e-06, 'wd': 3}. Best is trial 0 with value: 2.0231073777647555e+32.


Trial 0 finished with value: 2.0231073777647555e+32 and parameters: {'dls_func': 0, 'batch_size': 1, 'action_func': 0, 'loss_func': 0, 'metric': 0, 'gradient_clip': 0, 'max_norm': 12.626330626734708, 'one_cycle': 1, 'n_epoch': 0, 'lr': 7.357972600141196e-06, 'wd': 3}. Best is trial 0 with value: 2.0231073777647555e+32.
Trial 0 finished with value: 2.0231073777647555e+32 and parameters: {'dls_func': 0, 'batch_size': 1, 'action_func': 0, 'loss_func': 0, 'metric': 0, 'gradient_clip': 0, 'max_norm': 12.626330626734708, 'one_cycle': 1, 'n_epoch': 0, 'lr': 7.357972600141196e-06, 'wd': 3}. Best is trial 0 with value: 2.0231073777647555e+32.
Trial 0 finished with value: 2.0231073777647555e+32 and parameters: {'dls_func': 0, 'batch_size': 1, 'action_func': 0, 'loss_func': 0, 'metric': 0, 'gradient_clip': 0, 'max_norm': 12.626330626734708, 'one_cycle': 1, 'n_epoch': 0, 'lr': 7.357972600141196e-06, 'wd': 3}. Best is trial 0 with value: 2.0231073777647555e+32.
Trial 0 finished with value: 2.023107

epoch,train_loss,valid_loss,custom_mse,time
0,1246505728402853579456512.000000,202310718433662434272658486984704.000000,202310718433662434272658486984704.000000,00:00
1,1189930212847132726001664.000000,202310737776475548106725282283520.000000,202310737776475548106725282283520.000000,00:00
2,1211811870371036264398848.000000,202310718433662434272658486984704.000000,202310718433662434272658486984704.000000,00:00
3,1231222745052973291798528.000000,202310699090849320438591691685888.000000,202310699090849320438591691685888.000000,00:00
4,1216624452961641395388416.000000,202310699090849320438591691685888.000000,202310699090849320438591691685888.000000,00:00



*****************************************************************

Sampler:
**********ActionSampler**********
Action of func: action_dropout
Signature Vars:
Var of type: dropout_p
Configuration Space:
'low': 0.05, 'high': 0.95, 'log': False, 'step': 0.05
 Var of type: start_neurons
Configuration Space:
'choices' = [1, 2, 4, 8, 16, 32]
 Var of type: preprocess
Configuration Space:
'choices' = [True, False]
 Var of type: activation
Configuration Space:
'choices' = [<class 'torch.nn.modules.activation.ReLU'>, <class 'torch.nn.modules.activation.PReLU'>, <class 'torch.nn.modules.activation.SiLU'>]

Model loader parameters: dropout_p=0.55 start_neurons=16 preprocess=False activation=<class 'torch.nn.modules.activation.ReLU'>

Action of func: action_default
Signature Vars:
Var of type: start_neurons
Configuration Space:
'choices' = [2, 16]
 Var of type: preprocess
Configuration Space:
'choices' = [True, False]
 Var of type: activation
Configuration Space:
'choices' = [<class 'torch.nn.modul